In [31]:
%pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: psycopg2-binary in c:\users\gabriel panza\appdata\local\programs\python\python310\lib\site-packages (2.9.10)




[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [32]:
import psycopg2

In [33]:
# Conexão com o banco de dados padrão
connection = psycopg2.connect(
    dbname="postgres", user="postgres", password="1234", host="localhost"
)
connection.autocommit = True
cursor = connection.cursor()

In [34]:
# Desabilitar restrições temporariamente
cursor.execute("SET session_replication_role = 'replica';")

# Obter todas as tabelas e excluir uma por uma
cursor.execute("""
    SELECT tablename 
    FROM pg_tables 
    WHERE schemaname = 'public';
""")
tables = cursor.fetchall()

for table in tables:
    cursor.execute(f"DROP TABLE IF EXISTS {table[0]} CASCADE;")
    print(f"Tabela {table[0]} deletada com sucesso!")

# Reabilitar restrições
cursor.execute("SET session_replication_role = 'origin';")

# Deleto o Banco de Dados se existir
cursor.execute(f"DROP DATABASE IF EXISTS pokemon_db;")
# Crio o Banco de Dados
cursor.execute(f"CREATE DATABASE pokemon_db;")

print("Banco de dados criado com sucesso!")

Tabela ginasio deletada com sucesso!
Tabela insignia deletada com sucesso!
Tabela item_pokebola deletada com sucesso!
Tabela tipo deletada com sucesso!
Tabela golpe deletada com sucesso!
Tabela natureza deletada com sucesso!
Tabela item deletada com sucesso!
Tabela treinador deletada com sucesso!
Tabela pokemon deletada com sucesso!
Tabela item_evolucao deletada com sucesso!
Tabela item_batalha deletada com sucesso!
Tabela item_medicinais deletada com sucesso!
Tabela pokedex deletada com sucesso!
Tabela equipe deletada com sucesso!
Tabela liga deletada com sucesso!
Tabela batalha deletada com sucesso!
Tabela pode_aprender deletada com sucesso!
Banco de dados criado com sucesso!


In [35]:
# Criação das tabelas no banco de dados
sql_commands = [
    """
    CREATE TABLE Ginasio (
        id_ginasio SERIAL PRIMARY KEY,
        nome VARCHAR(50) NOT NULL,
        cidade VARCHAR(50) NOT NULL
    );
    """,
    """
    CREATE TABLE Insignia (
        id_insignia SERIAL PRIMARY KEY,
        id_ginasio INTEGER,
        nome VARCHAR(50) NOT NULL,
        descricao VARCHAR(100),
        FOREIGN KEY (id_ginasio) REFERENCES Ginasio(id_ginasio)
    );
    """,
    """
    CREATE TABLE Treinador (
        id_treinador SERIAL PRIMARY KEY,
        nome VARCHAR(50) NOT NULL,
        eh_campeao BOOLEAN,
        id_insignias INTEGER[],
        CHECK (array_length(id_insignias, 1) <= 8)
    );
    """,
    """
    CREATE TABLE Tipo (
        id_tipo SERIAL PRIMARY KEY,
        nome VARCHAR(50) NOT NULL
    );
    """,
    """
    CREATE TABLE Golpe (
        id_golpe SERIAL PRIMARY KEY,
        nome VARCHAR(50) NOT NULL,
        poder INTEGER,
        precisao INTEGER,
        id_tipo INTEGER,
        FOREIGN KEY (id_tipo) REFERENCES Tipo(id_tipo)
    );
    """,
    """
    CREATE TABLE Natureza (
        id_natureza SERIAL PRIMARY KEY,
        descricao VARCHAR(100) NOT NULL,
        efeito VARCHAR(100)
    );
    """,
    """
    CREATE TABLE Pokemon (
        id_pokedex_pokemon INTEGER,
        id_instancia_pokemon INTEGER,
        id_treinador INTEGER,
        nome VARCHAR(50) NOT NULL,
        nivel INTEGER,
        evolui_com_pedra BOOLEAN,
        id_tipo INTEGER[],
        condicao VARCHAR(20),
        hp INTEGER NOT NULL CHECK (hp >= 0),
        ataque INTEGER NOT NULL CHECK (ataque >= 0),
        defesa INTEGER NOT NULL CHECK (defesa >= 0),
        ataque_especial INTEGER NOT NULL CHECK (ataque_especial >= 0),
        defesa_especial INTEGER NOT NULL CHECK (defesa_especial >= 0),
        velocidade INTEGER NOT NULL CHECK (velocidade >= 0),
        id_natureza INTEGER,
        id_golpes INTEGER[],
        CHECK (array_length(id_golpes, 1) BETWEEN 1 AND 4),
        PRIMARY KEY (id_pokedex_pokemon, id_instancia_pokemon),
        FOREIGN KEY (id_treinador) REFERENCES Treinador(id_treinador)
    );
    """,
    """
    CREATE TABLE Item (
        id_item SERIAL PRIMARY KEY,
        nome VARCHAR(50) NOT NULL,
        descricao VARCHAR(100)
    );
    """,
    """
    CREATE TABLE Item_Evolucao (
        id_item INTEGER PRIMARY KEY,
        id_tipo INTEGER[],
        FOREIGN KEY (id_item) REFERENCES Item(id_item)
    );
    """,
    """
    CREATE TABLE Item_Batalha (
        id_item INTEGER PRIMARY KEY,
        valor_efeito_batalha INTEGER,
        FOREIGN KEY (id_item) REFERENCES Item(id_item)
    );
    """,
    """
    CREATE TABLE Item_Pokebola (
        id_item INTEGER PRIMARY KEY,
        taxa_captura FLOAT,
        FOREIGN KEY (id_item) REFERENCES Item(id_item)
    );
    """,
    """
    CREATE TABLE Item_Medicinais (
        id_item INTEGER PRIMARY KEY,
        valor_efeito_medicinal INTEGER,
        FOREIGN KEY (id_item) REFERENCES Item(id_item)
    );
    """,
    """
    CREATE TABLE Pokedex (
        id_pokedex SERIAL PRIMARY KEY,
        id_tipo INTEGER[],
        nome VARCHAR(30),
        descricao VARCHAR(200)
    );
    """,
    """
    CREATE TABLE Equipe (
        id_equipe SERIAL PRIMARY KEY,
        id_treinador INTEGER,
        id_pokemons INTEGER[],
        CHECK (array_length(id_pokemons, 1) BETWEEN 1 and 6),
        FOREIGN KEY (id_treinador) REFERENCES Treinador(id_treinador)
    );
    """,
    """
    CREATE TABLE Liga (
        id_membro SERIAL PRIMARY KEY,
        id_treinador INTEGER,
        nome VARCHAR(50) NOT NULL,
        especialidade VARCHAR(50) NOT NULL,
        descricao VARCHAR(200),
        FOREIGN KEY (id_treinador) REFERENCES Treinador(id_treinador)
    );
    """,
    """
    CREATE TABLE Pode_Aprender (
        id_pokedex_pokemon INTEGER,
        id_instancia_pokemon INTEGER,
        id_golpes INTEGER[],
        CHECK (array_length(id_golpes, 1) BETWEEN 4 and 30),
        PRIMARY KEY (id_pokedex_pokemon, id_instancia_pokemon),
        FOREIGN KEY (id_pokedex_pokemon, id_instancia_pokemon) REFERENCES Pokemon(id_pokedex_pokemon, id_instancia_pokemon)
    );
    """,
    """
    CREATE TABLE Batalha (
        id_treinador_1 INTEGER,
        id_treinador_2 INTEGER,
        vencedor INTEGER,
        PRIMARY KEY (id_treinador_1, id_treinador_2),
        FOREIGN KEY (id_treinador_1) REFERENCES Treinador(id_treinador),
        FOREIGN KEY (id_treinador_2) REFERENCES Treinador(id_treinador),
        FOREIGN KEY (vencedor) REFERENCES Treinador(id_treinador)
    );
    """
]

# Executando os comandos
for command in sql_commands:
    cursor.execute(command)

print("Tabelas do banco de dados criadas com sucesso!")

Tabelas do banco de dados criadas com sucesso!


In [36]:
# Inserções no banco de dados
sql_commands = [
"""
INSERT INTO Tipo (id_tipo, nome) VALUES
(1, 'Normal'),
(2, 'Fighting'),
(3, 'Flying'),
(4, 'Bug'),
(5, 'Ground'),
(6, 'Rock'),
(7, 'Ghost'),
(8, 'Poison'),
(10, 'Fire'),
(11, 'Water'),
(12, 'Grass'),
(13, 'Electric'),
(14, 'Psychic'),
(15, 'Ice'),
(16, 'Dragon');
""",
"""
INSERT INTO Pokedex (id_pokedex, id_tipo, nome, descricao) VALUES
(1, ARRAY[12], 'Bulbasaur', 'Uma semente estranha foi plantada em suas costas ao nascer. A planta cresce junto com o Pokémon.'),
(2, ARRAY[12], 'Ivysaur', 'Quando o bulbo em suas costas cresce, parece perder a capacidade de andar sobre as patas traseiras.'),
(3, ARRAY[12], 'Venusaur', 'A planta floresce quando absorve energia solar. Ele se move para buscar a luz do sol.'),
(4, ARRAY[10], 'Charmander', 'Prefere lugares quentes. Quando chove, é dito que vapor sai da ponta de sua cauda.'),
(5, ARRAY[10], 'Charmeleon', 'Quando balança sua cauda em chamas, a temperatura aumenta a níveis insuportáveis.'),
(6, ARRAY[10], 'Charizard', 'Cospe fogo quente o suficiente para derreter rochas. Pode causar incêndios florestais acidentalmente.'),
(7, ARRAY[11], 'Squirtle', 'Após nascer, suas costas incham e endurecem formando uma concha. Ele esguicha água pela boca.'),
(8, ARRAY[11], 'Wartortle', 'Geralmente se esconde na água para caçar presas desprevenidas. Move suas orelhas para manter o equilíbrio.'),
(9, ARRAY[11], 'Blastoise', 'Um Pokémon brutal com jatos de água de alta pressão em sua concha, usados para tackles rápidos.'),
(10, ARRAY[4], 'Caterpie', 'Suas patas possuem ventosas que permitem escalar superfícies inclinadas e paredes sem esforço.'),
(11, ARRAY[4], 'Metapod', 'Este Pokémon é vulnerável enquanto sua casca está mole, expondo seu corpo frágil.'),
(12, ARRAY[4,3], 'Butterfree', 'Em batalha, bate suas asas rapidamente para liberar poeira tóxica no ar.'),
(13, ARRAY[4], 'Weedle', 'Frequentemente encontrado em florestas, comendo folhas. Possui um ferrão venenoso na cabeça.'),
(14, ARRAY[4], 'Kakuna', 'Quase incapaz de se mover, endurece sua casca para se proteger dos predadores.'),
(15, ARRAY[4,3], 'Beedrill', 'Voa rapidamente e ataca com seus ferrões venenosos nas patas dianteiras e cauda.'),
(16, ARRAY[1,3], 'Pidgey', 'Comum em florestas e bosques. Bate as asas no solo para levantar areia cegante.'),
(17, ARRAY[1,3], 'Pidgeotto', 'Muito protetor de seu território, ataca ferozmente intrusos com bicadas.'),
(18, ARRAY[1,3], 'Pidgeot', 'Caça sobrevoando a água em alta velocidade para pegar presas como Magikarp.'),
(19, ARRAY[1], 'Rattata', 'Morde qualquer coisa quando ataca. Pequeno e muito rápido, é visto em muitos lugares.'),
(20, ARRAY[1], 'Raticate', 'Usa seus bigodes para manter o equilíbrio. Parece desacelerar quando eles são cortados.'),
(21, ARRAY[1,3], 'Spearow', 'Come insetos em áreas gramadas. Suas asas curtas precisam bater rapidamente para mantê-lo no ar.'),
(22, ARRAY[1,3], 'Fearow', 'Com suas asas enormes, pode se manter no ar por longos períodos sem precisar pousar.'),
(23, ARRAY[8], 'Ekans', 'Move-se silenciosamente. Come ovos de pássaros como Pidgey e Spearow inteiros.'),
(24, ARRAY[8], 'Arbok', 'Os padrões ameaçadores em sua barriga variam de região para região.'),
(25, ARRAY[13], 'Pikachu', 'Quando vários Pikachu se reúnem, sua eletricidade pode causar tempestades.'),
(26, ARRAY[13], 'Raichu', 'Sua cauda longa serve como aterramento para protegê-lo de sua própria eletricidade.'),
(27, ARRAY[5], 'Sandshrew', 'Escava profundamente em áreas áridas e longe da água. Só emerge para caçar comida.'),
(28, ARRAY[5], 'Sandslash', 'Enrola-se em uma bola espinhosa quando ameaçado. Pode rolar nessa forma para atacar.'),
(29, ARRAY[8], 'Nidoran♀', 'Apesar de pequeno, seus espinhos venenosos o tornam perigoso. A fêmea tem chifres menores.'),
(30, ARRAY[8], 'Nidorina', 'O chifre da fêmea se desenvolve lentamente. Prefere ataques físicos como arranhões e mordidas.'),
(31, ARRAY[8,5], 'Nidoqueen', 'Suas escamas duras proporcionam grande proteção. Usa seu corpo robusto para golpes poderosos.'),
(32, ARRAY[8], 'Nidoran♂', 'Endurece suas orelhas para detectar perigo. Quanto maiores os chifres, mais poderoso o veneno.'),
(33, ARRAY[8], 'Nidorino', 'Um Pokémon agressivo e rápido para atacar. Seu chifre excreta um veneno poderoso.'),
(34, ARRAY[8,5], 'Nidoking', 'Usa sua cauda poderosa para lutar, esmagar e quebrar os ossos da presa.'),
(35, ARRAY[1], 'Clefairy', 'Seu charme mágico atrai muitos admiradores. É raro e encontrado apenas em certas áreas.'),
(36, ARRAY[1], 'Clefable', 'Um Pokémon fada tímido que raramente é visto. Foge ao primeiro sinal de pessoas.'),
(37, ARRAY[10], 'Vulpix', 'Ao nascer, tem apenas uma cauda. A cauda se divide conforme envelhece.'),
(38, ARRAY[10], 'Ninetales', 'Muito inteligente e vingativo. Pegar uma de suas caudas pode resultar em uma maldição de 1000 anos.'),
(39, ARRAY[1], 'Jigglypuff', 'Quando seus olhos brilham, canta uma melodia relaxante que adormece os oponentes.'),
(40, ARRAY[1], 'Wigglytuff', 'Seu corpo é macio e flexível. Quando irritado, infla-se a um tamanho enorme.'),
(41, ARRAY[8,3], 'Zubat', 'Forma colônias em lugares escuros. Usa ondas ultrassônicas para localizar presas.'),
(42, ARRAY[8,3], 'Golbat', 'Depois de atacar, não para de sugar a energia da vítima, mesmo se ficar pesado demais para voar.'),
(43, ARRAY[12,8], 'Oddish', 'Durante o dia, mantém o rosto enterrado no solo. À noite, anda espalhando suas sementes.'),
(44, ARRAY[12,8], 'Gloom', 'O fluido que escorre de sua boca não é baba, mas néctar usado para atrair presas.'),
(45, ARRAY[12,8], 'Vileplume', 'Quanto maiores suas pétalas, mais pólen tóxico contêm. Sua cabeça grande é pesada e difícil de sustentar.'),
(46, ARRAY[12,4], 'Paras', 'Enterra-se para sugar raízes de árvores. Os cogumelos crescem em suas costas, extraindo nutrientes do Pokémon.'),
(47, ARRAY[12,4], 'Parasect', 'Um par hospedeiro-parasita onde o cogumelo tomou conta do corpo do inseto. Prefere locais úmidos.'),
(48, ARRAY[12,4], 'Venonat', 'Vive nas sombras de árvores altas, onde se alimenta de insetos.'),
(49, ARRAY[12,4], 'Venomoth', 'Ativo à noite, come insetos atraídos pela luz. Suas asas são cobertas de escamas tóxicas.'),
(50, ARRAY[5], 'Diglett', 'Vive a cerca de um metro de profundidade no solo, onde se alimenta de raízes de plantas.'),
(51, ARRAY[5], 'Dugtrio', 'Uma equipe de três Digletts cavando em uníssono pode causar terremotos.'),
(52, ARRAY[1], 'Meowth', 'Ama coisas redondas e brilhantes. As noites passam caçando objetos brilhantes para colecionar.'),
(53, ARRAY[1], 'Persian', 'Muito orgulhoso de seu pelo fino e brilhante. Ele anda elegante como se estivesse desfilando.'),
(54, ARRAY[11], 'Psyduck', 'É frequentemente incomodado por dores de cabeça. Quando as dores aumentam, ele libera poderes psíquicos.'),
(55, ARRAY[11], 'Golduck', 'Conhecido por suas habilidades de natação, ele nada rapidamente até em águas turbulentas.'),
(56, ARRAY[2], 'Mankey', 'Muito irritadiço. Quando começa a se enfurecer, não consegue parar.'),
(57, ARRAY[2], 'Primeape', 'Entra em fúria tão intensa que não consegue distinguir amigos de inimigos.'),
(58, ARRAY[10], 'Growlithe', 'Extremamente leal. Protegerá seu treinador até o fim, mesmo enfrentando oponentes maiores.'),
(59, ARRAY[10], 'Arcanine', 'Um Pokémon lendário famoso por sua velocidade e nobreza. Diz-se que cruza continentes em um dia.'),
(60, ARRAY[11], 'Poliwag', 'Sua pele é tão fina que se podem ver seus órgãos. O espiral no seu estômago é, na verdade, seus intestinos.'),
(61, ARRAY[11], 'Poliwhirl', 'Seu corpo escorregadio o protege de ser agarrado durante batalhas.'),
(62, ARRAY[11,2], 'Poliwrath', 'Suas poderosas pernas permitem nadar grandes distâncias com facilidade.'),
(63, ARRAY[14], 'Abra', 'Passa 18 horas do dia dormindo. Enquanto dorme, usa seus poderes psíquicos para se proteger.'),
(64, ARRAY[14], 'Kadabra', 'Diz-se que uma criança com poderes psíquicos despertou um dia e se transformou em Kadabra.'),
(65, ARRAY[14], 'Alakazam', 'Suas habilidades psíquicas são incríveis. Ele pode se lembrar de cada detalhe de sua vida inteira.'),
(66, ARRAY[2], 'Machop', 'Ama treinar seus músculos e pode levantar 1000 pessoas ao mesmo tempo.'),
(67, ARRAY[2], 'Machoke', 'Seus músculos são tão poderosos que precisa de um cinto especial para conter sua força.'),
(68, ARRAY[2], 'Machamp', 'Com quatro braços poderosos, pode realizar múltiplos socos ao mesmo tempo.'),
(69, ARRAY[12,8], 'Bellsprout', 'Prefere ambientes úmidos e quentes. Se alimenta de pequenos insetos usando sua boca.'),
(70, ARRAY[12,8], 'Weepinbell', 'Libera um pó tóxico de sua boca. Se se sente ameaçado, libera a substância instantaneamente.'),
(71, ARRAY[12,8], 'Victreebel', 'Atrai presas com um aroma doce e as dissolve lentamente com ácido.'),
(72, ARRAY[11,8], 'Tentacool', 'Flutua perto da superfície do mar. Se sua pele secar, ele retornará ao oceano.'),
(73, ARRAY[11,8], 'Tentacruel', 'Com seus tentáculos, pode capturar e imobilizar a presa facilmente.'),
(74, ARRAY[6,5], 'Geodude', 'Costuma ser confundido com rochas. É encontrado frequentemente em montanhas e cavernas.'),
(75, ARRAY[6,5], 'Graveler', 'Desce colinas rolando e quebrando tudo em seu caminho.'),
(76, ARRAY[6,5], 'Golem', 'Seu corpo é tão duro quanto uma rocha. Pode sobreviver até a uma erupção vulcânica.'),
(77, ARRAY[10], 'Ponyta', 'Suas pernas ficam fortes ao correr por planícies. É capaz de saltar sobre edifícios altos.'),
(78, ARRAY[10], 'Rapidash', 'Pode correr a uma velocidade incrível, suas chamas brilham quando atinge a máxima velocidade.'),
(79, ARRAY[11,14], 'Slowpoke', 'Muito lento e distraído. Às vezes, esquece o que estava fazendo.'),
(80, ARRAY[11,14], 'Slowbro', 'Quando um Shellder morde sua cauda, ele se torna mais forte e focado.'),
(81, ARRAY[13], 'Magnemite', 'Gera campos magnéticos poderosos. Fica preso a objetos metálicos.'),
(82, ARRAY[13], 'Magneton', 'Um grupo de três Magnemite. Sua presença pode causar interferências elétricas.'),
(83, ARRAY[1,3], 'Farfetch’d', 'Usa o talo de alho-poró como arma. É raro e muito difícil de encontrar na natureza.'),
(84, ARRAY[1,3], 'Doduo', 'Possui duas cabeças e pode correr rapidamente. Ambas as cabeças se alternam para vigiar.'),
(85, ARRAY[1,3], 'Dodrio', 'Com três cabeças, pode realizar ataques coordenados. Suas pernas são incrivelmente rápidas.'),
(86, ARRAY[11], 'Seel', 'Adora nadar em águas frias. Seu chifre afiado pode quebrar gelo com facilidade.'),
(87, ARRAY[11,15], 'Dewgong', 'Nada graciosamente em águas geladas. Sua pele é isolante e protege do frio intenso.'),
(88, ARRAY[8], 'Grimer', 'Nascido de lodo tóxico. Deixa um rastro contaminado por onde passa.'),
(89, ARRAY[8], 'Muk', 'É feito de uma substância tóxica que pode poluir rios e lagos.'),
(90, ARRAY[11], 'Shellder', 'Sua concha é extremamente dura. Protege seu corpo macio dos predadores.'),
(91, ARRAY[11,15], 'Cloyster', 'Pode se fechar completamente, tornando-se quase impenetrável.'),
(92, ARRAY[7,8], 'Gastly', 'Feito de um gás venenoso. Pode atravessar paredes e superfícies sólidas.'),
(93, ARRAY[7,8], 'Haunter', 'Flutua no ar e pode causar paralisia apenas tocando a vítima.'),
(94, ARRAY[7,8], 'Gengar', 'Adora pregar peças. Às vezes, se esconde nas sombras para assustar pessoas.'),
(95, ARRAY[6,5], 'Onix', 'Seus segmentos de pedra formam um corpo longo e resistente.'),
(96, ARRAY[14], 'Drowzee', 'Pode induzir o sono e se alimenta dos sonhos de seus alvos.'),
(97, ARRAY[14], 'Hypno', 'Usa seu pêndulo para hipnotizar suas presas e induzir o sono.'),
(98, ARRAY[11], 'Krabby', 'Vive em praias e margens de rios. Usa suas garras para se proteger.'),
(99, ARRAY[11], 'Kingler', 'Sua garra gigante é tão pesada que às vezes dificulta seu equilíbrio.'),
(100, ARRAY[13], 'Voltorb', 'Frequentemente confundido com uma Pokébola. Explode ao ser tocado.'),
(101, ARRAY[13], 'Electrode', 'Acumula energia elétrica e pode explodir de repente.'),
(102, ARRAY[12,14], 'Exeggcute', 'Consiste em seis ovos que se comunicam telepaticamente.'),
(103, ARRAY[12,14], 'Exeggutor', 'Suas cabeças falam independentemente, mas cooperam em grupo.'),
(104, ARRAY[5], 'Cubone', 'Usa o osso de sua mãe falecida como arma. É um Pokémon muito solitário.'),
(105, ARRAY[5], 'Marowak', 'Superou a dor da perda. Usa o osso para executar ataques poderosos.'),
(106, ARRAY[2], 'Hitmonlee', 'Especialista em chutes. Suas pernas podem se estender para ataques de longo alcance.'),
(107, ARRAY[2], 'Hitmonchan', 'Especialista em socos. Suas técnicas de boxe são incríveis.'),
(108, ARRAY[1], 'Lickitung', 'Sua língua pegajosa pode se estender até o dobro do tamanho de seu corpo.'),
(109, ARRAY[8], 'Koffing', 'Expele gases tóxicos. É encontrado em locais poluídos.'),
(110, ARRAY[8], 'Weezing', 'Formado por dois Koffing fundidos. Seus gases são ainda mais tóxicos.'),
(111, ARRAY[5,6], 'Rhyhorn', 'Seu corpo blindado é capaz de resistir a impactos poderosos.'),
(112, ARRAY[5,6], 'Rhydon', 'Seus chifres perfuram até rochas. Sua pele é resistente como armadura.'),
(113, ARRAY[1], 'Chansey', 'Um Pokémon gentil que compartilha ovos nutritivos para ajudar os feridos.'),
(114, ARRAY[12], 'Tangela', 'Coberto por vinhas azuis. Ninguém jamais viu seu verdadeiro corpo.'),
(115, ARRAY[1], 'Kangaskhan', 'Cuida do filhote em sua bolsa. Extremamente protetora e feroz.'),
(116, ARRAY[11], 'Horsea', 'Libera jatos de tinta quando está em perigo para escapar.'),
(117, ARRAY[11], 'Seadra', 'Navega pelas correntes marinhas. Suas nadadeiras são afiadas como lâminas.'),
(118, ARRAY[11], 'Goldeen', 'Nada elegantemente, suas barbatanas se movem como um vestido fluido.'),
(119, ARRAY[11], 'Seaking', 'Durante a estação de acasalamento, nadam juntos em padrões hipnotizantes.'),
(120, ARRAY[11], 'Staryu', 'Mesmo se perder uma das pontas de seu corpo em forma de estrela, pode regenerá-la.'),
(121, ARRAY[11, 14], 'Starmie', 'Suas gemas centrais brilham de forma mística. Comunica-se usando luz piscante.'),
(122, ARRAY[14], 'Mr. Mime', 'Especialista em mímica. Cria barreiras invisíveis imitando gestos.'),
(123, ARRAY[4,3], 'Scyther', 'Seus braços afiados como lâminas podem cortar oponentes com facilidade.'),
(124, ARRAY[15,14], 'Jynx', 'Dança de forma hipnótica para confundir os oponentes durante a batalha.'),
(125, ARRAY[13], 'Electabuzz', 'A eletricidade em seu corpo causa interferências em aparelhos eletrônicos.'),
(126, ARRAY[10], 'Magmar', 'O calor que emana de seu corpo pode derreter pedras e incendiar florestas.'),
(127, ARRAY[4], 'Pinsir', 'Aperta seus inimigos com suas pinças poderosas até que parem de se mover.'),
(128, ARRAY[1], 'Tauros', 'Corre por campos enquanto chicoteia o próprio corpo com suas caudas.'),
(129, ARRAY[11], 'Magikarp', 'Extremamente fraco e inútil, mas pode pular incrivelmente alto.'),
(130, ARRAY[11,3], 'Gyarados', 'Após evoluir, torna-se violento e destrutivo, frequentemente atacando cidades.'),
(131, ARRAY[11], 'Lapras', 'Gentil e inteligente, adora carregar pessoas em suas costas pelos mares.'),
(132, ARRAY[1], 'Ditto', 'Pode se transformar em qualquer objeto ou Pokémon, copiando sua aparência.'),
(133, ARRAY[1], 'Eevee', 'Possui uma estrutura genética instável, permitindo evoluções variadas.'),
(134, ARRAY[11], 'Vaporeon', 'Sua pele se mistura com a água, tornando-se praticamente invisível.'),
(135, ARRAY[13], 'Jolteon', 'Gera eletricidade estática e pode disparar agulhas de seus pelos.'),
(136, ARRAY[10], 'Flareon', 'Seu corpo armazena calor intenso, capaz de exalar chamas a 900°C.'),
(137, ARRAY[1], 'Porygon', 'Um Pokémon artificial criado por humanos através de programação de computador.'),
(138, ARRAY[6,11], 'Omanyte', 'Fóssil restaurado. Possui tentáculos usados para nadar.'),
(139, ARRAY[6,11], 'Omastar', 'Seus tentáculos são usados para capturar presas e se mover lentamente.'),
(140, ARRAY[6,11], 'Kabuto', 'Reviveu de um fóssil. Costuma se esconder na areia em águas rasas.'),
(141, ARRAY[6,11], 'Kabutops', 'Seus braços afiados como lâminas são usados para cortar presas.'),
(142, ARRAY[6,3], 'Aerodactyl', 'Reviveu de um fóssil. Conhecido por seu voo rápido e mordidas poderosas.'),
(143, ARRAY[1], 'Snorlax', 'Passa o dia dormindo e comendo. Difícil de acordar quando está em sono profundo.'),
(144, ARRAY[15,3], 'Articuno', 'Lendário pássaro de gelo. Diz-se que traz o inverno ao bater as asas.'),
(145, ARRAY[13,3], 'Zapdos', 'Lendário pássaro elétrico. Suas asas soltam faíscas ao bater.'),
(146, ARRAY[10,3], 'Moltres', 'Lendário pássaro de fogo. Suas asas brilham intensamente em chamas.'),
(147, ARRAY[16,3], 'Dratini', 'Raro e místico, vive escondido em profundezas aquáticas.'),
(148, ARRAY[16], 'Dragonair', 'Seu corpo longo e elegante emite uma aura misteriosa.'),
(149, ARRAY[16], 'Dragonite', 'Pode voar ao redor do mundo em um dia. Muito inteligente e amigável.'),
(150, ARRAY[14], 'Mewtwo', 'Criado geneticamente. Seu poder psíquico é inigualável.'),
(151, ARRAY[14], 'Mew', 'Dizem ser o ancestral de todos os Pokémon. Possui DNA de cada espécie.');
""",
"""
INSERT INTO Natureza (id_natureza, descricao, efeito) VALUES
(1, 'Hardy', 'Neutro'),
(2, 'Lonely', '+Ataque,-Defesa'),
(3, 'Brave', '+Ataque,-Velocidade'),
(4, 'Adamant', '+Ataque,-Ataque Especial'),
(5, 'Naughty', '+Ataque,-Defesa Especial'),
(6, 'Bold', '+Defesa,-Ataque'),
(7, 'Docile', 'Neutro'),
(8, 'Relaxed', '+Defesa,-Velocidade'),
(9, 'Impish', '+Defesa,-Ataque Especial'),
(10, 'Lax', '+Defesa,-Defesa Especial'),
(11, 'Timid', '+Velocidade,-Ataque'),
(12, 'Hasty', '+Velocidade,-Defesa'),
(13, 'Serious', 'Neutro'),
(14, 'Jolly', '+Velocidade,-Ataque Especial'),
(15, 'Naive', '+Velocidade,-Defesa Especial'),
(16, 'Modest', '+Ataque Especial,-Ataque'),
(17, 'Mild', '+Ataque Especial,-Defesa'),
(18, 'Quiet', '+Ataque Especial,-Velocidade'),
(19, 'Bashful', 'Neutro'),
(20, 'Rash', '+Ataque Especial,-Defesa Especial'),
(21, 'Calm', '+Defesa Especial,-Ataque'),
(22, 'Gentle', '+Defesa Especial,-Defesa'),
(23, 'Sassy', '+Defesa Especial,-Velocidade'),
(24, 'Careful', '+Defesa Especial,-Ataque Especial'),
(25, 'Quirky', 'Neutro');
""",
"""
INSERT INTO Golpe (id_golpe, nome, poder, precisao, id_tipo) VALUES
(1, 'Tackle', 40, 100, 1), -- Normal
(2, 'Vine Whip', 45, 100, 12), -- Grass
(3, 'Ember', 40, 100, 10), -- Fire
(4, 'Water Gun', 40, 100, 11), -- Water
(5, 'Thunder Shock', 40, 100, 13), -- Electric
(6, 'Gust', 40, 100, 3), -- Flying
(7, 'Quick Attack', 40, 100, 1), -- Normal
(8, 'Scratch', 40, 100, 1), -- Normal
(9, 'Razor Leaf', 55, 95, 12), -- Grass
(10, 'Flamethrower', 90, 100, 10), -- Fire
(11, 'Hydro Pump', 110, 80, 11), -- Water
(12, 'Thunderbolt', 90, 100, 13), -- Electric
(13, 'Hyper Beam', 150, 90, 1), -- Normal
(14, 'Earthquake', 100, 100, 5), -- Ground
(15, 'Rock Slide', 75, 90, 6), -- Rock
(16, 'Ice Beam', 90, 100, 15), -- Ice
(17, 'Blizzard', 110, 70, 15), -- Ice
(18, 'Confusion', 50, 100, 14), -- Psychic
(19, 'Psychic', 90, 100, 14), -- Psychic
(20, 'Leech Seed', 0, 90, 12), -- Grass
(21, 'Fire Spin', 35, 85, 10), -- Fire
(22, 'Bubble Beam', 65, 100, 11), -- Water
(23, 'Thunder Wave', 0, 90, 13), -- Electric
(24, 'Poison Sting', 15, 100, 8), -- Poison
(25, 'Acid', 40, 100, 8), -- Poison
(26, 'Sludge', 65, 100, 8), -- Poison
(27, 'Body Slam', 85, 100, 1), -- Normal
(28, 'Double-Edge', 120, 100, 1), -- Normal
(29, 'Wrap', 15, 90, 1), -- Normal
(30, 'Horn Attack', 65, 100, 1), -- Normal
(31, 'Stomp', 65, 100, 1), -- Normal
(32, 'Egg Bomb', 100, 75, 1), -- Normal
(33, 'Peck', 35, 100, 3), -- Flying
(34, 'Drill Peck', 80, 100, 3), -- Flying
(35, 'Aurora Beam', 65, 100, 15), -- Ice
(36, 'Sky Attack', 140, 90, 3), -- Flying
(37, 'Mega Punch', 80, 85, 1), -- Normal
(38, 'Mega Kick', 120, 75, 1), -- Normal
(39, 'Horn Drill', NULL, 30, 1), -- Normal
(40, 'Seismic Toss', NULL, 100, 2), -- Fighting
(41, 'Counter', NULL, 100, 2), -- Fighting
(42, 'Karate Chop', 50, 100, 2), -- Fighting
(43, 'Submission', 80, 80, 2), -- Fighting
(44, 'Low Kick', 50, 90, 2), -- Fighting
(45, 'Night Shade', NULL, 100, 7), -- Ghost
(46, 'Lick', 30, 100, 7), -- Ghost
(47, 'Shadow Punch', 60, 100, 7), -- Ghost
(48, 'Sludge Bomb', 90, 100, 8), -- Poison
(49, 'Dream Eater', 100, 100, 14), -- Psychic
(50, 'Self-Destruct', 200, 100, 1), -- Normal
(51, 'Explosion', 250, 100, 1), -- Normal
(52, 'Spore', NULL, 100, 12), -- Grass
(53, 'Sleep Powder', NULL, 75, 12), -- Grass
(54, 'Stun Spore', NULL, 75, 12), -- Grass
(55, 'Toxic', NULL, 90, 8), -- Poison
(56, 'Haze', NULL, NULL, 15), -- Ice
(57, 'Confuse Ray', NULL, 100, 7), -- Ghost
(58, 'Barrier', NULL, NULL, 14), -- Psychic
(59, 'Light Screen', NULL, NULL, 14), -- Psychic
(60, 'Reflect', NULL, NULL, 14), -- Psychic
(61, 'Swift', 60, NULL, 1), -- Normal
(62, 'Rock Throw', 50, 90, 6), -- Rock
(63, 'Leech Life', 80, 100, 4), -- Bug
(64, 'String Shot', NULL, 95, 4), -- Bug
(65, 'Rage', 20, 100, 1), -- Normal
(66, 'Bide', NULL, NULL, 1), -- Normal
(67, 'Mirror Move', NULL, NULL, 3), -- Flying
(68, 'Transform', NULL, NULL, 1), -- Normal
(69, 'Metronome', NULL, NULL, 1), -- Normal
(70, 'Swords Dance', NULL, NULL, 1), -- Normal
(71, 'Growth', NULL, NULL, 12), -- Grass
(72, 'Harden', NULL, NULL, 1), -- Normal
(73, 'Minimize', NULL, NULL, 1), -- Normal
(74, 'Double Team', NULL, NULL, 1), -- Normal
(75, 'Recover', NULL, NULL, 1), -- Normal
(76, 'Soft-Boiled', NULL, NULL, 1), -- Normal
(77, 'Focus Energy', NULL, NULL, 1), -- Normal
(78, 'Barrage', 15, 85, 1), -- Normal
(79, 'Clamp', 35, 85, 11), -- Water
(80, 'Constrict', 10, 100, 1), -- Normal
(81, 'Dragon Breath', 60, 100, 16); -- Dragon
""",
"""
INSERT INTO Golpe (id_golpe, nome, poder, precisao, id_tipo) VALUES
(104, 'TM09 Take Down', 90, 85, 1),
(122, 'TM23 Dragon Rage', 40, 100, 16),
(123, 'TM25 Thunder', 110, 70, 13),
(127, 'TM28 Dig', 80, 100, 5),
(128, 'TM29 Psychic', 90, 100, 14),
(129, 'TM30 Teleport', NULL, NULL, 14),
(137, 'TM38 Fire Blast', 110, 85, 10),
(143, 'TM44 Rest', NULL, NULL, 1),
(145, 'TM46 Psywave', NULL, 80, 14);
""",
"""
INSERT INTO Ginasio (id_ginasio, nome, cidade) VALUES
(1, 'Ginásio de Pewter', 'Pewter City'),
(2, 'Ginásio de Cerulean', 'Cerulean City'),
(3, 'Ginásio de Vermilion', 'Vermilion City'),
(4, 'Ginásio de Celadon', 'Celadon City'),
(5, 'Ginásio de Fuchsia', 'Fuchsia City'),
(6, 'Ginásio de Saffron', 'Saffron City'),
(7, 'Ginásio de Cinnabar', 'Cinnabar Island'),
(8, 'Ginásio de Viridian', 'Viridian City');
""",
"""
INSERT INTO Insignia (id_insignia, id_ginasio, nome, descricao) VALUES
(1, 1, 'Boulder Badge', 'Insígnia da Cidade de Pewter, conquistada ao derrotar Brock.'),
(2, 2, 'Cascade Badge', 'Insígnia da Cidade de Cerulean, conquistada ao derrotar Misty.'),
(3, 3, 'Thunder Badge', 'Insígnia da Cidade de Vermilion, conquistada ao derrotar Lt. Surge.'),
(4, 4, 'Rainbow Badge', 'Insígnia da Cidade de Celadon, conquistada ao derrotar Erika.'),
(5, 5, 'Soul Badge', 'Insígnia da Cidade de Fuchsia, conquistada ao derrotar Koga.'),
(6, 6, 'Marsh Badge', 'Insígnia da Cidade de Saffron, conquistada ao derrotar Sabrina.'),
(7, 7, 'Volcano Badge', 'Insígnia da Ilha Cinnabar, conquistada ao derrotar Blaine.'),
(8, 8, 'Earth Badge', 'Insígnia da Cidade de Viridian, conquistada ao derrotar Giovanni.');
""",
"""
INSERT INTO Treinador (id_treinador, nome, eh_campeao, id_insignias) VALUES
(1, 'Jotave', FALSE, ARRAY[1, 2, 3, 4, 5, 6, 7, 8]),
(2, 'Panza', FALSE, ARRAY[1, 2, 3, 4, 5, 6, 7, 8]),
(3, 'Barbara', FALSE, ARRAY[1, 2, 3, 4, 5, 6, 7, 8]),
(4, 'Daniel', FALSE, ARRAY[1, 2, 3, 4, 5, 6, 7, 8]),
(5, 'Raquel', FALSE, ARRAY[1, 2, 3, 4, 5, 6, 7]),
(6, 'Lorelei', FALSE, ARRAY[1, 2, 3, 4, 5, 6, 7, 8]),
(7, 'Bruno', FALSE, ARRAY[1, 2, 3, 4, 5, 6, 7, 8]),
(8, 'Agatha', FALSE, ARRAY[1, 2, 3, 4, 5, 6, 7, 8]),
(9, 'Lance', FALSE, ARRAY[1, 2, 3, 4, 5, 6, 7, 8]);
""",
"""
INSERT INTO Liga (id_membro, id_treinador, nome, especialidade, descricao) VALUES
(1, 6, 'Lorelei', 'Ice', 'Especialista em Pokémon do tipo Gelo e Água. Primeira treinadora da Liga.'),
(2, 7, 'Bruno', 'Fighting', 'Especialista em Pokémon do tipo Lutador e Rocha. Segundo desafiante da Liga.'),
(3, 8, 'Agatha', 'Ghost', 'Especialista em Pokémon do tipo Fantasma e Veneno. Terceira treinadora da Liga.'),
(4, 9, 'Lance', 'Dragon', 'Especialista em Pokémon do tipo Dragão e Voador. Último membro da Elite antes do Campeão.');
""",
"""
INSERT INTO Item (id_item, nome, descricao) VALUES
(1, 'Potion', 'Restaura 20 pontos de vida de um Pokémon.'),
(2, 'Super Potion', 'Restaura 50 pontos de vida de um Pokémon.'),
(3, 'Hyper Potion', 'Restaura 200 pontos de vida de um Pokémon.'),
(4, 'Max Potion', 'Restaura todo o HP de um Pokémon.'),
(5, 'Full Restore', 'Restaura todo o HP e remove todas as condições de status.'),
(6, 'Antidote', 'Cura envenenamento de um Pokémon.'),
(7, 'Paralyze Heal', 'Cura paralisia de um Pokémon.'),
(8, 'Burn Heal', 'Cura queimaduras de um Pokémon.'),
(9, 'Ice Heal', 'Descongela um Pokémon.'),
(10, 'Awakening', 'Acorda um Pokémon adormecido.'),
(11, 'Revive', 'Revive um Pokémon desmaiado com metade do HP.'),
(12, 'Max Revive', 'Revive um Pokémon desmaiado com HP completo.'),
(13, 'Poké Ball', 'Usada para capturar Pokémon selvagens.'),
(14, 'Great Ball', 'Tem uma taxa de captura melhor que a Poké Ball.'),
(15, 'Ultra Ball', 'Tem uma taxa de captura melhor que a Great Ball.'),
(16, 'Master Ball', 'Captura qualquer Pokémon sem falhar.'),
(17, 'Rare Candy', 'Aumenta o nível de um Pokémon em 1.'),
(18, 'Thunder Stone', 'Evolui Pikachu e Eevee.'),
(19, 'Water Stone', 'Evolui Poliwhirl, Shellder e Eevee.'),
(20, 'Fire Stone', 'Evolui Eevee, Vulpix e Growlithe.'),
(21, 'Leaf Stone', 'Evolui Gloom, Weepinbell e Exeggcute.'),
(22, 'Moon Stone', 'Evolui Clefairy, Nidorina e Nidorino.'),
(23, 'X Attack', 'Aumenta o Ataque em uma batalha.'),
(24, 'X Defense', 'Aumenta a Defesa em uma batalha.'),
(25, 'X Speed', 'Aumenta a Velocidade em uma batalha.'),
(26, 'X Special', 'Aumenta o Ataque Especial em uma batalha.');
""",
"""
INSERT INTO Item_Medicinais (id_item, valor_efeito_medicinal) VALUES
(1, 20),    -- Potion
(2, 50),    -- Super Potion
(3, 200),   -- Hyper Potion
(4, 9999),  -- Max Potion
(5, 9999),  -- Full Restore
(6, 0),     -- Antidote
(7, 0),     -- Paralyze Heal
(8, 0),     -- Burn Heal
(9, 0),     -- Ice Heal
(10, 0),    -- Awakening
(11, 50),   -- Revive
(12, 9999); -- Max Revive
""",
"""
INSERT INTO Item_Pokebola (id_item, taxa_captura) VALUES
(13, 1.0),    -- Poké Ball
(14, 1.5),    -- Great Ball
(15, 2.0),    -- Ultra Ball
(16, 255.0);  -- Master Ball (garante captura)
""",
"""
INSERT INTO Item_Evolucao (id_item, id_tipo) VALUES
(17, NULL),  -- Rare Candy
(18, ARRAY[13]),  -- Thunder Stone
(19, ARRAY[11]),  -- Water Stone
(20, ARRAY[10]),  -- Fire Stone
(21, ARRAY[12]),  -- Leaf Stone
(22, ARRAY[1,8]);  -- Moon Stone
""",
"""
INSERT INTO Item_Batalha (id_item, valor_efeito_batalha) VALUES
(23, 1),   -- X Attack (Aumenta ataque)
(24, 1),   -- X Defense (Aumenta defesa)
(25, 1),   -- X Speed (Aumenta velocidade)
(26, 1);   -- X Special (Aumenta ataque especial)
""",
"""
-- Inserção de instâncias de Pokémon para Jotave
INSERT INTO Pokemon (id_pokedex_pokemon, id_instancia_pokemon, id_treinador, nome, nivel, evolui_com_pedra, id_tipo, condicao, hp, ataque, defesa, ataque_especial, defesa_especial, velocidade, id_natureza, id_golpes)
VALUES 
(6, 1, 1, 'Charizard', 50, FALSE, ARRAY[10,3], 'Saudável', 150, 90, 85, 110, 85, 100, 4, ARRAY[3,10,7,104]),
(25, 1, 1, 'Pikachu', 45, TRUE, ARRAY[13], 'Saudável', 110, 55, 50, 75, 60, 110, 11, ARRAY[5,12,23,7]),
(2, 1, 1, 'Ivysaur', 45, FALSE, ARRAY[12], 'Saudável', 130, 80, 90, 95, 85, 70, 5, ARRAY[2,9,20,7]),
(45, 1, 1, 'Vileplume', 40, FALSE, ARRAY[12,8], 'Saudável', 140, 90, 100, 110, 90, 50, 8, ARRAY[2,9,20,7]),
(77, 1, 1, 'Ponyta', 42, FALSE, ARRAY[10], 'Saudável', 105, 85, 80, 75, 60, 90, 2, ARRAY[3,10,7,104]),
(12, 1, 1, 'Butterfree', 48, FALSE, ARRAY[4,3], 'Saudável', 100, 60, 70, 90, 70, 85, 3, ARRAY[6,33,7,34]);
""",
"""
-- Inserção de instâncias de Pokémon para Panza
INSERT INTO Pokemon (id_pokedex_pokemon, id_instancia_pokemon, id_treinador, nome, nivel, evolui_com_pedra, id_tipo, condicao, hp, ataque, defesa, ataque_especial, defesa_especial, velocidade, id_natureza, id_golpes)
VALUES 
(149, 1, 2, 'Dragonite', 55, FALSE, ARRAY[16,3], 'Saudável', 180, 120, 110, 100, 100, 80, 4, ARRAY[81,122,123,104]),
(3, 1, 2, 'Venusaur', 50, FALSE, ARRAY[12], 'Saudável', 170, 90, 95, 110, 100, 70, 6, ARRAY[2,9,20,7]),
(20, 1, 2, 'Raticate', 40, FALSE, ARRAY[1], 'Saudável', 105, 75, 70, 60, 60, 120, 7, ARRAY[1,7,27,30]),
(50, 1, 2, 'Diglett', 45, FALSE, ARRAY[5], 'Saudável', 75, 80, 60, 50, 40, 130, 2, ARRAY[14,4,31,7]),
(90, 1, 2, 'Shellder', 45, TRUE, ARRAY[11], 'Saudável', 115, 70, 90, 60, 85, 50, 3, ARRAY[4,11,22,7]),
(130, 1, 2, 'Gyarados', 55, FALSE, ARRAY[11,3], 'Saudável', 190, 110, 100, 80, 80, 70, 4, ARRAY[6,33,7,34]);
""",
"""
-- Inserção de instâncias de Pokémon para Barbara (time rosa)
INSERT INTO Pokemon (id_pokedex_pokemon, id_instancia_pokemon, id_treinador, nome, nivel, evolui_com_pedra, id_tipo, condicao, hp, ataque, defesa, ataque_especial, defesa_especial, velocidade, id_natureza, id_golpes)
VALUES 
(36, 1, 3, 'Clefable', 40, FALSE, ARRAY[1], 'Saudável', 150, 70, 85, 95, 90, 60, 1, ARRAY[1,7,27,30]),
(39, 1, 3, 'Jigglypuff', 38, FALSE, ARRAY[1], 'Saudável', 120, 60, 60, 70, 70, 50, 5, ARRAY[1,7,27,30]),
(40, 1, 3, 'Wigglytuff', 45, FALSE, ARRAY[1], 'Saudável', 140, 75, 70, 80, 75, 60, 6, ARRAY[1,7,27,30]),
(113, 1, 3, 'Chansey', 48, FALSE, ARRAY[1], 'Saudável', 200, 60, 70, 90, 90, 50, 3, ARRAY[1,7,27,30]),
(80, 1, 3, 'Slowbro', 42, FALSE, ARRAY[11,14], 'Saudável', 160, 80, 100, 110, 100, 40, 2, ARRAY[4,11,22,7]),
(108, 1, 3, 'Lickitung', 40, FALSE, ARRAY[1], 'Saudável', 130, 70, 85, 70, 75, 50, 4, ARRAY[1,7,27,30]);
""",
"""
-- Inserção de instâncias de Pokémon para Daniel (competitivamente fortes)
INSERT INTO Pokemon (id_pokedex_pokemon, id_instancia_pokemon, id_treinador, nome, nivel, evolui_com_pedra, id_tipo, condicao, hp, ataque, defesa, ataque_especial, defesa_especial, velocidade, id_natureza, id_golpes)
VALUES 
(65, 1, 4, 'Alakazam', 55, FALSE, ARRAY[14], 'Saudável', 130, 60, 70, 150, 90, 120, 3, ARRAY[18,19,7,14]),
(130, 2, 4, 'Gyarados', 55, FALSE, ARRAY[11,3], 'Saudável', 190, 110, 100, 80, 80, 70, 6, ARRAY[6,33,7,34]),
(143, 1, 4, 'Snorlax', 50, FALSE, ARRAY[1], 'Saudável', 210, 100, 100, 70, 90, 30, 4, ARRAY[1,7,27,30]),
(135, 1, 4, 'Jolteon', 50, FALSE, ARRAY[13], 'Saudável', 130, 85, 70, 110, 90, 130, 5, ARRAY[5,12,23,7]),
(94, 1, 4, 'Gengar', 52, FALSE, ARRAY[7,8], 'Saudável', 140, 90, 80, 130, 85, 110, 2, ARRAY[24,7,25,26]),
(121, 1, 4, 'Starmie', 50, FALSE, ARRAY[11,14], 'Saudável', 150, 80, 90, 120, 100, 100, 3, ARRAY[4,11,22,7]);
""",
"""
-- Inserção de instâncias de Pokémon para Raquel (fracos contra Giovanni)
INSERT INTO Pokemon (id_pokedex_pokemon, id_instancia_pokemon, id_treinador, nome, nivel, evolui_com_pedra, id_tipo, condicao, hp, ataque, defesa, ataque_especial, defesa_especial, velocidade, id_natureza, id_golpes)
VALUES 
(25, 2, 5, 'Pikachu', 30, TRUE, ARRAY[13], 'Saudável', 100, 55, 50, 75, 60, 110, 11, ARRAY[5,12,23,7]),
(37, 1, 5, 'Vulpix', 38, TRUE, ARRAY[10], 'Saudável', 95, 70, 65, 75, 60, 90, 2, ARRAY[3,10,7,104]),
(126, 1, 5, 'Magmar', 35, FALSE, ARRAY[10], 'Saudável', 120, 90, 85, 90, 85, 80, 5, ARRAY[3,10,7,104]),
(109, 1, 5, 'Koffing', 32, FALSE, ARRAY[8], 'Saudável', 100, 60, 70, 80, 80, 50, 4, ARRAY[24,25,26,29]),
(101, 1, 5, 'Electrode', 30, FALSE, ARRAY[13], 'Saudável', 100, 70, 60, 80, 60, 150, 3, ARRAY[5,12,23,7]),
(58, 1, 5, 'Growlithe', 38, TRUE, ARRAY[10], 'Saudável', 105, 85, 80, 75, 60, 90, 2, ARRAY[3,10,7,104]);
""",
"""
-- Inserção de instâncias de Pokémon para Lorelei (especialista em Gelo e Água)
INSERT INTO Pokemon (id_pokedex_pokemon, id_instancia_pokemon, id_treinador, nome, nivel, evolui_com_pedra, id_tipo, condicao, hp, ataque, defesa, ataque_especial, defesa_especial, velocidade, id_natureza, id_golpes)
VALUES 
(87, 1, 6, 'Dewgong', 56, FALSE, ARRAY[11,15], 'Saudável', 180, 80, 100, 90, 120, 70, 3, ARRAY[4,11,22,7]),
(91, 1, 6, 'Cloyster', 54, FALSE, ARRAY[11,15], 'Saudável', 150, 95, 180, 85, 110, 70, 5, ARRAY[4,11,22,7]),
(80, 2, 6, 'Slowbro', 55, FALSE, ARRAY[11,14], 'Saudável', 190, 75, 110, 100, 120, 30, 2, ARRAY[18,19,7,14]),
(124, 1, 6, 'Jynx', 56, FALSE, ARRAY[15,14], 'Saudável', 170, 60, 70, 120, 90, 95, 1, ARRAY[15,16,18,19]),
(131, 1, 6, 'Lapras', 56, FALSE, ARRAY[11,15], 'Saudável', 200, 85, 90, 95, 110, 80, 6, ARRAY[4,11,22,7]);
""",
"""
-- Inserção de instâncias de Pokémon para Bruno (especialista em Lutador e Rocha)
INSERT INTO Pokemon (id_pokedex_pokemon, id_instancia_pokemon, id_treinador, nome, nivel, evolui_com_pedra, id_tipo, condicao, hp, ataque, defesa, ataque_especial, defesa_especial, velocidade, id_natureza, id_golpes)
VALUES 
(95, 1, 7, 'Onix', 53, FALSE, ARRAY[6,5], 'Saudável', 160, 70, 180, 50, 85, 40, 3, ARRAY[14,4,31,7]),
(95, 2, 7, 'Onix', 53, FALSE, ARRAY[6,5], 'Saudável', 160, 70, 180, 50, 85, 40, 3, ARRAY[14,4,31,7]),
(66, 1, 7, 'Machop', 55, FALSE, ARRAY[2], 'Saudável', 180, 100, 85, 70, 85, 60, 2, ARRAY[42,43,40,41]),
(67, 1, 7, 'Machoke', 56, FALSE, ARRAY[2], 'Saudável', 190, 110, 90, 80, 85, 55, 2, ARRAY[42,43,40,41]),
(68, 1, 7, 'Machamp', 58, FALSE, ARRAY[2], 'Saudável', 200, 120, 100, 85, 90, 50, 1, ARRAY[42,43,40,41]);
""",
"""
-- Inserção de instâncias de Pokémon para Agatha (especialista em Fantasma e Veneno)
INSERT INTO Pokemon (id_pokedex_pokemon, id_instancia_pokemon, id_treinador, nome, nivel, evolui_com_pedra, id_tipo, condicao, hp, ataque, defesa, ataque_especial, defesa_especial, velocidade, id_natureza, id_golpes)
VALUES 
(94, 2, 8, 'Gengar', 56, FALSE, ARRAY[7,8], 'Saudável', 150, 75, 80, 130, 90, 110, 5, ARRAY[24,7,25,26]),
(42, 1, 8, 'Golbat', 55, FALSE, ARRAY[8,3], 'Saudável', 170, 85, 90, 80, 80, 100, 4, ARRAY[24,7,25,26]),
(93, 1, 8, 'Haunter', 55, FALSE, ARRAY[7,8], 'Saudável', 140, 75, 85, 120, 85, 100, 3, ARRAY[24,7,25,26]),
(24, 1, 8, 'Arbok', 56, FALSE, ARRAY[8], 'Saudável', 170, 85, 95, 70, 85, 95, 6, ARRAY[24,7,25,26]),
(94, 3, 8, 'Gengar', 58, FALSE, ARRAY[7,8], 'Saudável', 160, 80, 85, 140, 100, 120, 2, ARRAY[24,7,25,26]);
""",
"""
-- Inserção de instâncias de Pokémon para Lance (especialista em Dragão)
INSERT INTO Pokemon (id_pokedex_pokemon, id_instancia_pokemon, id_treinador, nome, nivel, evolui_com_pedra, id_tipo, condicao, hp, ataque, defesa, ataque_especial, defesa_especial, velocidade, id_natureza, id_golpes)
VALUES 
(148, 1, 9, 'Dragonair', 56, FALSE, ARRAY[16], 'Saudável', 170, 95, 100, 85, 85, 90, 5, ARRAY[81,122,123,104]),
(148, 2, 9, 'Dragonair', 56, FALSE, ARRAY[16], 'Saudável', 170, 85, 85, 95, 100, 90, 4, ARRAY[81,122,123,104]),
(130, 3, 9, 'Gyarados', 58, FALSE, ARRAY[11,3], 'Saudável', 200, 120, 100, 90, 85, 80, 3, ARRAY[6,33,7,34]),
(142, 1, 9, 'Aerodactyl', 58, FALSE, ARRAY[6,3], 'Saudável', 180, 105, 85, 70, 80, 130, 2, ARRAY[6,33,7,34]),
(149, 2, 9, 'Dragonite', 60, FALSE, ARRAY[16,3], 'Saudável', 220, 120, 110, 100, 100, 80, 1, ARRAY[81,122,123,104]);
""",
"""
-- Inserção de instâncias para os Pokémons ainda não inseridos + Dragonair do Panza
INSERT INTO Pokemon (id_pokedex_pokemon, id_instancia_pokemon, id_treinador, nome, nivel, evolui_com_pedra, id_tipo, condicao, hp, ataque, defesa, ataque_especial, defesa_especial, velocidade, id_natureza, id_golpes)
VALUES 
(1, 1, NULL, 'Bulbasaur', 5, FALSE, ARRAY[12], 'Saudável', 20, 50, 50, 65, 60, 45, 1, ARRAY[2,9,20,7]),
(4, 1, 1, 'Charmander', 5, FALSE, ARRAY[10], 'Saudável', 20, 52, 43, 60, 50, 65, 2, ARRAY[3,10,7,104]),
(5, 1, 1, 'Charmeleon', 5, FALSE, ARRAY[10], 'Saudável', 20, 52, 43, 60, 50, 65, 2, ARRAY[3,10,7,104]),
(7, 1, NULL, 'Squirtle', 5, FALSE, ARRAY[11], 'Saudável', 20, 48, 65, 50, 64, 43, 3, ARRAY[4,11,22,7]),
(8, 1, NULL, 'Wartortle', 5, FALSE, ARRAY[11], 'Saudável', 20, 48, 65, 50, 64, 43, 3, ARRAY[4,11,22,7]),
(9, 1, NULL, 'Blastoise', 5, FALSE, ARRAY[11], 'Saudável', 20, 83, 100, 85, 105, 78, 2, ARRAY[4,11,22,7]),
(10, 1, NULL, 'Caterpie', 5, FALSE, ARRAY[4], 'Saudável', 20, 30, 35, 20, 20, 45, 4, ARRAY[6,33,7,34]),
(11, 1, NULL, 'Metapod', 5, FALSE, ARRAY[4], 'Saudável', 20, 20, 55, 25, 25, 30, 30, ARRAY[6,33,7,34]),
(13, 1, NULL, 'Weedle', 5, FALSE, ARRAY[4,8], 'Saudável', 20, 35, 30, 20, 20, 50, 5, ARRAY[24,25,26,29]),
(16, 1, NULL, 'Pidgey', 5, FALSE, ARRAY[1,3], 'Saudável', 20, 45, 40, 35, 35, 56, 6, ARRAY[6,33,7,34]),
(19, 1, NULL, 'Rattata', 5, FALSE, ARRAY[1], 'Saudável', 20, 56, 35, 25, 35, 72, 7, ARRAY[1,7,27,30]),
(21, 1, NULL, 'Spearow', 5, FALSE, ARRAY[1,3], 'Saudável', 20, 60, 30, 31, 31, 70, 8, ARRAY[6,33,7,34]),
(23, 1, NULL, 'Ekans', 5, FALSE, ARRAY[8], 'Saudável', 20, 60, 44, 40, 54, 55, 9, ARRAY[24,25,26,29]),
(27, 1, NULL, 'Sandshrew', 5, FALSE, ARRAY[5], 'Saudável', 20, 75, 85, 20, 30, 40, 10, ARRAY[14,4,31,7]),
(29, 1, NULL, 'Nidoran♀', 5, FALSE, ARRAY[8], 'Saudável', 20, 47, 52, 40, 40, 41, 11, ARRAY[24,25,26,29]),
(32, 1, NULL, 'Nidoran♂', 5, FALSE, ARRAY[8], 'Saudável', 20, 57, 40, 40, 40, 50, 12, ARRAY[24,25,26,29]),
(35, 1, 3, 'Clefairy', 5, TRUE, ARRAY[1], 'Saudável', 20, 45, 48, 60, 65, 35, 13, ARRAY[1,7,27,30]),
(41, 1, NULL, 'Zubat', 5, FALSE, ARRAY[8,3], 'Saudável', 20, 45, 35, 30, 40, 55, 15, ARRAY[24,25,26,29]),
(43, 1, NULL, 'Oddish', 5, FALSE, ARRAY[12,8], 'Saudável', 20, 50, 55, 75, 65, 30, 16, ARRAY[2,9,20,7]),
(46, 1, NULL, 'Paras', 5, FALSE, ARRAY[12,4], 'Saudável', 20, 70, 55, 45, 55, 25, 17, ARRAY[2,9,20,7]),
(48, 1, NULL, 'Venonat', 5, FALSE, ARRAY[12,4], 'Saudável', 20, 55, 50, 40, 55, 45, 18, ARRAY[2,9,20,7]),
(52, 1, NULL, 'Meowth', 5, FALSE, ARRAY[1], 'Saudável', 20, 45, 35, 40, 40, 90, 20, ARRAY[1,7,27,30]),
(54, 1, NULL, 'Psyduck', 5, FALSE, ARRAY[11], 'Saudável', 20, 52, 48, 65, 50, 55, 21, ARRAY[4,11,22,7]),
(56, 1, NULL, 'Mankey', 5, FALSE, ARRAY[2], 'Saudável', 20, 80, 35, 35, 45, 70, 22, ARRAY[42,43,40,41]),
(60, 1, NULL, 'Poliwag', 5, TRUE, ARRAY[11], 'Saudável', 20, 50, 40, 40, 40, 90, 24, ARRAY[4,11,22,7]),
(63, 1, NULL, 'Abra', 5, FALSE, ARRAY[14], 'Saudável', 20, 20, 15, 105, 55, 90, 25, ARRAY[18,19,7,14]),
(69, 1, NULL, 'Bellsprout', 5, TRUE, ARRAY[12,8], 'Saudável', 20, 75, 35, 70, 30, 40, 24, ARRAY[2,9,20,7]),
(72, 1, NULL, 'Tentacool', 5, FALSE, ARRAY[11,8], 'Saudável', 20, 40, 35, 50, 100, 70, 23, ARRAY[4,11,22,7]),
(74, 1, NULL, 'Geodude', 5, FALSE, ARRAY[6,5], 'Saudável', 20, 80, 100, 30, 30, 20, 22, ARRAY[14,4,31,7]),
(81, 1, NULL, 'Magnemite', 5, FALSE, ARRAY[13,17], 'Saudável', 20, 35, 70, 95, 55, 45, 1, ARRAY[5,12,23,7]),
(84, 1, NULL, 'Doduo', 5, FALSE, ARRAY[1,3], 'Saudável', 20, 85, 45, 35, 35, 75, 2, ARRAY[6,33,7,34]),
(86, 1, NULL, 'Seel', 5, FALSE, ARRAY[11], 'Saudável', 20, 45, 55, 45, 70, 45, 3, ARRAY[4,11,22,7]),
(98, 1, NULL, 'Krabby', 5, FALSE, ARRAY[11], 'Saudável', 20, 105, 90, 25, 25, 50, 4, ARRAY[4,11,22,7]),
(104, 1, NULL, 'Cubone', 5, FALSE, ARRAY[5], 'Saudável', 20, 50, 95, 40, 50, 35, 5, ARRAY[14,4,31,7]),
(111, 1, NULL, 'Rhyhorn', 5, FALSE, ARRAY[6,5], 'Saudável', 20, 85, 95, 30, 30, 25, 6, ARRAY[14,4,31,7]),
(116, 1, NULL, 'Horsea', 5, FALSE, ARRAY[11], 'Saudável', 20, 40, 70, 70, 70, 60, 7, ARRAY[4,11,22,7]),
(120, 1, NULL, 'Staryu', 5, FALSE, ARRAY[11], 'Saudável', 20, 45, 55, 70, 55, 85, 8, ARRAY[4,11,22,7]),
(123, 1, NULL, 'Scyther', 5, FALSE, ARRAY[4,3], 'Saudável', 20, 110, 80, 55, 80, 105, 16, ARRAY[6,33,7,34]),
(127, 1, NULL, 'Pinsir', 5, FALSE, ARRAY[4], 'Saudável', 20, 125, 100, 55, 70, 85, 9, ARRAY[2,9,20,7]),
(137, 1, NULL, 'Porygon', 5, FALSE, ARRAY[1], 'Saudável', 20, 60, 70, 85, 75, 40, 10, ARRAY[1,7,27,30]),
(138, 1, NULL, 'Omanyte', 5, FALSE, ARRAY[6,11], 'Saudável', 20, 40, 100, 90, 55, 35, 18, ARRAY[4,11,22,7]),
(139, 1, NULL, 'Omastar', 5, FALSE, ARRAY[6,11], 'Saudável', 20, 60, 125, 115, 70, 55, 19, ARRAY[4,11,22,7]),
(140, 1, NULL, 'Kabuto', 5, FALSE, ARRAY[6,11], 'Saudável', 20, 80, 90, 55, 45, 55, 20, ARRAY[4,11,22,7]),
(141, 1, NULL, 'Kabutops', 5, FALSE, ARRAY[6,11], 'Saudável', 20, 115, 105, 65, 70, 80, 21, ARRAY[4,11,22,7]),
(144, 1, NULL, 'Articuno', 5, FALSE, ARRAY[15,3], 'Saudável', 20, 85, 100, 95, 125, 85, 23, ARRAY[4,11,22,7]),
(145, 1, NULL, 'Zapdos', 5, FALSE, ARRAY[13,3], 'Saudável', 20, 90, 85, 125, 90, 100, 24, ARRAY[5,12,23,7]),
(146, 1, NULL, 'Moltres', 5, FALSE, ARRAY[10,3], 'Saudável', 20, 100, 90, 125, 85, 90, 25, ARRAY[3,10,7,104]),
(147, 1, 2, 'Dratini', 5, FALSE, ARRAY[16], 'Saudável', 20, 64, 45, 50, 50, 50, 11, ARRAY[81,122,123,104]),
(150, 1, NULL, 'Mewtwo', 50, FALSE, ARRAY[14], 'Saudável', 100, 110, 90, 154, 90, 130, 12, ARRAY[18,19,7,14]),
(151, 1, NULL, 'Mew', 50, FALSE, ARRAY[14], 'Saudável', 100, 100, 100, 100, 100, 100, 13, ARRAY[18,19,7,14]),
(14, 1, NULL, 'Kakuna', 5, FALSE, ARRAY[4,8], 'Saudável', 20, 30, 55, 25, 25, 30, 21, ARRAY[6,33,7,34]),
(15, 1, NULL, 'Beedrill', 5, FALSE, ARRAY[4,8], 'Saudável', 20, 80, 40, 45, 80, 75, 5, ARRAY[24,25,26,29]),
(17, 1, NULL, 'Pidgeotto', 5, FALSE, ARRAY[1,3], 'Saudável', 20, 60, 55, 50, 50, 71, 6, ARRAY[6,33,7,34]),
(18, 1, NULL, 'Pidgeot', 5, FALSE, ARRAY[1,3], 'Saudável', 20, 80, 70, 65, 70, 101, 7, ARRAY[6,33,7,34]),
(22, 1, NULL, 'Fearow', 5, FALSE, ARRAY[1,3], 'Saudável', 20, 90, 65, 60, 60, 100, 9, ARRAY[6,33,7,34]),
(28, 1, NULL, 'Sandslash', 5, FALSE, ARRAY[5], 'Saudável', 20, 100, 75, 50, 50, 65, 11, ARRAY[14,4,31,7]),
(31, 1, NULL, 'Nidoqueen', 5, FALSE, ARRAY[8,5], 'Saudável', 20, 90, 90, 75, 85, 76, 12, ARRAY[24,25,26,29]),
(34, 1, NULL, 'Nidoking', 5, FALSE, ARRAY[8,5], 'Saudável', 20, 102, 77, 85, 75, 85, 13, ARRAY[24,25,26,29]),
(38, 1, NULL, 'Ninetales', 5, FALSE, ARRAY[10], 'Saudável', 20, 81, 75, 80, 100, 100, 14, ARRAY[3,10,7,104]),
(47, 1, NULL, 'Parasect', 5, FALSE, ARRAY[12,4], 'Saudável', 20, 95, 80, 60, 80, 30, 15, ARRAY[2,9,20,7]),
(49, 1, NULL, 'Venomoth', 5, FALSE, ARRAY[4,8], 'Saudável', 20, 70, 65, 90, 75, 90, 16, ARRAY[24,25,26,29]),
(51, 1, NULL, 'Dugtrio', 5, FALSE, ARRAY[5], 'Saudável', 20, 80, 50, 70, 60, 120, 17, ARRAY[14,4,31,7]),
(53, 1, NULL, 'Persian', 5, FALSE, ARRAY[1], 'Saudável', 20, 70, 60, 65, 65, 115, 18, ARRAY[1,7,27,30]),
(55, 1, NULL, 'Golduck', 5, FALSE, ARRAY[11], 'Saudável', 20, 82, 78, 95, 80, 85, 19, ARRAY[4,11,22,7]),
(57, 1, NULL, 'Primeape', 5, FALSE, ARRAY[2], 'Saudável', 20, 105, 60, 70, 70, 95, 20, ARRAY[42,43,40,41]),
(59, 1, NULL, 'Arcanine', 5, FALSE, ARRAY[10], 'Saudável', 20, 110, 80, 100, 80, 95, 21, ARRAY[3,10,7,104]),
(61, 1, NULL, 'Poliwhirl', 5, TRUE, ARRAY[11], 'Saudável', 20, 65, 65, 50, 50, 90, 22, ARRAY[4,11,22,7]),
(64, 1, NULL, 'Kadabra', 5, FALSE, ARRAY[14], 'Saudável', 20, 35, 30, 120, 70, 105, 23, ARRAY[18,19,7,14]),
(70, 1, NULL, 'Weepinbell', 5, FALSE, ARRAY[12,8], 'Saudável', 20, 90, 50, 85, 50, 55, 24, ARRAY[2,9,20,7]),
(75, 1, NULL, 'Graveler', 5, FALSE, ARRAY[6,5], 'Saudável', 20, 95, 115, 45, 45, 35, 25, ARRAY[14,4,31,7]),
(82, 1, NULL, 'Magneton', 5, FALSE, ARRAY[13,17], 'Saudável', 20, 60, 95, 120, 70, 70, 20, ARRAY[5,12,23,7]),
(83, 1, NULL, 'Farfetch’d', 5, FALSE, ARRAY[1,3], 'Saudável', 20, 90, 55, 85, 85, 60, 19, ARRAY[6,33,7,34]),
(92, 1, NULL, 'Gastly', 5, FALSE, ARRAY[7,8], 'Saudável', 20, 35, 30, 100, 35, 80, 18, ARRAY[24,7,25,26]),
(117, 1, NULL, 'Seadra', 5, FALSE, ARRAY[11], 'Saudável', 20, 65, 70, 95, 80, 85, 17, ARRAY[4,11,22,7]),
(118, 1, NULL, 'Goldeen', 5, FALSE, ARRAY[11], 'Saudável', 20, 67, 60, 45, 50, 63, 16, ARRAY[4,11,22,7]),
(26, 1, NULL, 'Raichu', 5, FALSE, ARRAY[13], 'Saudável', 20, 90, 55, 90, 80, 110, 3, ARRAY[5,12,23,7]),
(30, 1, NULL, 'Nidorina', 5, TRUE, ARRAY[8], 'Saudável', 20, 70, 70, 55, 55, 56, 4, ARRAY[24,25,26,29]),
(33, 1, NULL, 'Nidorino', 5, TRUE, ARRAY[8], 'Saudável', 20, 72, 57, 55, 55, 65, 5, ARRAY[24,25,26,29]),
(44, 1, NULL, 'Gloom', 5, FALSE, ARRAY[12,8], 'Saudável', 20, 65, 70, 85, 85, 40, 7, ARRAY[2,9,20,7]),
(73, 1, NULL, 'Tentacruel', 5, FALSE, ARRAY[11,8], 'Saudável', 20, 70, 65, 120, 100, 100, 10, ARRAY[4,11,22,7]),
(76, 1, NULL, 'Golem', 5, FALSE, ARRAY[6,5], 'Saudável', 20, 120, 130, 55, 65, 45, 11, ARRAY[14,4,31,7]),
(79, 1, NULL, 'Slowpoke', 5, FALSE, ARRAY[11,14], 'Saudável', 20, 65, 65, 40, 40, 15, 12, ARRAY[4,11,22,7]),
(85, 1, NULL, 'Dodrio', 5, FALSE, ARRAY[1,3], 'Saudável', 20, 110, 70, 60, 60, 110, 13, ARRAY[6,33,7,34]),
(88, 1, NULL, 'Grimer', 5, FALSE, ARRAY[8], 'Saudável', 20, 80, 50, 40, 50, 25, 14, ARRAY[24,25,26,29]),
(89, 1, NULL, 'Muk', 5, FALSE, ARRAY[8], 'Saudável', 20, 105, 75, 65, 65, 50, 15, ARRAY[24,25,26,29]),
(97, 1, NULL, 'Hypno', 5, FALSE, ARRAY[14], 'Saudável', 20, 85, 70, 115, 100, 67, 18, ARRAY[18,19,7,14]),
(99, 1, NULL, 'Kingler', 5, FALSE, ARRAY[11], 'Saudável', 20, 130, 115, 50, 50, 75, 19, ARRAY[4,11,22,7]),
(100, 1, NULL, 'Voltorb', 5, FALSE, ARRAY[13], 'Saudável', 20, 40, 50, 55, 50, 100, 20, ARRAY[5,12,23,7]),
(102, 1, NULL, 'Exeggcute', 5, TRUE, ARRAY[12,14], 'Saudável', 20, 90, 80, 120, 95, 50, 20, ARRAY[2,9,20,7]),
(103, 1, NULL, 'Exeggutor', 5, FALSE, ARRAY[12,14], 'Saudável', 20, 95, 85, 125, 100, 55, 21, ARRAY[2,9,20,7]),
(105, 1, NULL, 'Marowak', 5, FALSE, ARRAY[5], 'Saudável', 20, 80, 110, 50, 80, 45, 22, ARRAY[14,4,31,7]),
(107, 1, NULL, 'Hitmonchan', 5, FALSE, ARRAY[2], 'Saudável', 20, 105, 79, 110, 80, 76, 23, ARRAY[42,43,40,41]),
(110, 1, NULL, 'Weezing', 5, FALSE, ARRAY[8], 'Saudável', 20, 90, 120, 85, 85, 60, 24, ARRAY[24,25,26,29]),
(112, 1, NULL, 'Rhydon', 5, FALSE, ARRAY[6,5], 'Saudável', 20, 130, 120, 55, 60, 40, 25, ARRAY[14,4,31,7]),
(114, 1, NULL, 'Tangela', 5, FALSE, ARRAY[12], 'Saudável', 20, 55, 115, 100, 90, 60, 14, ARRAY[2,9,20,7]),
(115, 1, NULL, 'Kangaskhan', 5, FALSE, ARRAY[1], 'Saudável', 20, 105, 80, 40, 80, 90, 13, ARRAY[1,7,27,30]),
(119, 1, NULL, 'Seaking', 5, FALSE, ARRAY[11], 'Saudável', 20, 92, 65, 65, 80, 68, 12, ARRAY[4,11,22,7]),
(122, 1, NULL, 'Mr. Mime', 5, FALSE, ARRAY[14], 'Saudável', 20, 50, 65, 120, 100, 90, 11, ARRAY[18,19,7,14]),
(125, 1, NULL, 'Electabuzz', 5, FALSE, ARRAY[13], 'Saudável', 20, 83, 75, 95, 85, 105, 10, ARRAY[5,12,23,7]),
(132, 1, NULL, 'Ditto', 5, FALSE, ARRAY[1], 'Saudável', 20, 48, 48, 48, 48, 48, 9, ARRAY[1,7,27,30]),
(62, 1, NULL, 'Poliwrath ', 5, FALSE, ARRAY[1], 'Saudável', 20, 48, 48, 48, 48, 48, 8, ARRAY[1,7,27,30]),
(71, 1, NULL, 'Victreebel ', 5, FALSE, ARRAY[1], 'Saudável', 20, 48, 48, 48, 48, 48, 7, ARRAY[1,7,27,30]),
(78, 1, NULL, 'Rapidash ', 5, FALSE, ARRAY[1], 'Saudável', 20, 48, 48, 48, 48, 48, 6, ARRAY[1,7,27,30]),
(96, 1, NULL, 'Drowzee', 5, FALSE, ARRAY[1], 'Saudável', 20, 48, 48, 48, 48, 48, 5, ARRAY[1,7,27,30]),
(106, 1, NULL, 'Hitmonlee', 5, FALSE, ARRAY[1], 'Saudável', 20, 48, 48, 48, 48, 48, 4, ARRAY[1,7,27,30]),
(128, 1, NULL, 'Tauros', 5, FALSE, ARRAY[1], 'Saudável', 20, 48, 48, 48, 48, 48, 3, ARRAY[1,7,27,30]),
(129, 1, NULL, 'Magikarp', 5, FALSE, ARRAY[1], 'Saudável', 20, 48, 48, 48, 48, 48, 2, ARRAY[1,7,27,30]),
(133, 1, NULL, 'Eevee', 5, TRUE, ARRAY[1], 'Saudável', 20, 48, 48, 48, 48, 48, 1, ARRAY[1,7,27,30]),
(134, 1, NULL, 'Vaporeon', 5, FALSE, ARRAY[1], 'Saudável', 20, 48, 48, 48, 48, 48, 12, ARRAY[1,7,27,30]),
(136, 1, NULL, 'Flareon', 5, FALSE, ARRAY[1], 'Saudável', 20, 48, 48, 48, 48, 48, 13, ARRAY[1,7,27,30]),
(148, 3, 2, 'Dragonair', 56, FALSE, ARRAY[16], 'Saudável', 170, 100, 90, 100, 90, 85, 2, ARRAY[81,122,123,104]);
""",
"""
INSERT INTO Pode_Aprender (id_pokedex_pokemon, id_instancia_pokemon, id_golpes) VALUES
(1, 1, ARRAY[1, 2, 20, 7, 104, 121, 105, 123]),     -- Bulbasaur
(2, 1, ARRAY[1, 2, 20, 7, 104, 121, 105, 123]),     -- Ivysaur
(3, 1, ARRAY[1, 2, 20, 7, 104, 121, 105, 123]),     -- Venusaur
(4, 1, ARRAY[8, 3, 10, 7, 104, 137, 100, 104]),     -- Charmander
(5, 1, ARRAY[8, 3, 10, 7, 104, 137, 100, 104]),     -- Charmeleon
(6, 1, ARRAY[8, 3, 10, 7, 104, 137, 100, 104]),     -- Charizard
(7, 1, ARRAY[4, 11, 22, 7, 123]),    	  -- Squirtle
(8, 1, ARRAY[4, 11, 22, 7, 123]),    	  -- Wartortle
(9, 1, ARRAY[4, 11, 22, 7, 123]),    	  -- Blastoise
(10, 1, ARRAY[1, 2, 20, 7, 121]),    	  -- Caterpie
(11, 1, ARRAY[1, 2, 20, 7, 121]),    	  -- Metapod
(12, 1, ARRAY[1, 2, 20, 7, 53]),     	  -- Butterfree
(13, 1, ARRAY[24, 25, 26, 7, 123]),  	  -- Weedle
(14, 1, ARRAY[24, 25, 26, 7, 123]),  	  -- Kakuna
(15, 1, ARRAY[24, 25, 26, 7, 123]),  	  -- Beedrill
(16, 1, ARRAY[6, 33, 7, 34, 104]),        -- Pidgey
(17, 1, ARRAY[6, 33, 7, 34, 104]),        -- Pidgeotto
(18, 1, ARRAY[6, 33, 7, 34, 104]),        -- Pidgeot
(19, 1, ARRAY[1, 7, 27, 30, 114]),        -- Rattata
(20, 1, ARRAY[1, 7, 27, 30, 114]),        -- Raticate
(21, 1, ARRAY[6, 33, 7, 34, 104]),        -- Spearow
(22, 1, ARRAY[6, 33, 7, 34, 104]),        -- Fearow
(23, 1, ARRAY[24, 25, 26, 29, 123]),      -- Ekans
(24, 1, ARRAY[24, 25, 26, 29, 123]),      -- Arbok
(25, 1, ARRAY[5, 12, 23, 7, 123]),        -- Pikachu
(26, 1, ARRAY[5, 12, 23, 7, 123]),        -- Raichu
(27, 1, ARRAY[14, 4, 31, 7, 127]),        -- Sandshrew
(28, 1, ARRAY[14, 4, 31, 7, 127]),        -- Sandslash
(29, 1, ARRAY[24, 25, 26, 7, 104]),       -- Nidoran♀
(30, 1, ARRAY[24, 25, 26, 7, 104]),       -- Nidorina
(31, 1, ARRAY[24, 25, 26, 7, 104]),       -- Nidoqueen
(32, 1, ARRAY[24, 25, 26, 7, 104]),       -- Nidoran♂
(33, 1, ARRAY[24, 25, 26, 7, 104]),       -- Nidorino
(34, 1, ARRAY[24, 25, 26, 7, 104]),       -- Nidoking
(35, 1, ARRAY[1, 7, 27, 30, 104]),        -- Clefairy
(36, 1, ARRAY[1, 7, 27, 30, 104]),        -- Clefable
(37, 1, ARRAY[3, 10, 7, 14, 104, 137]),        -- Vulpix
(38, 1, ARRAY[3, 10, 7, 14, 104, 137]),        -- Ninetales
(39, 1, ARRAY[1, 7, 27, 30, 104]),             -- Jigglypuff
(40, 1, ARRAY[1, 7, 27, 30, 104]),             -- Wigglytuff
(41, 1, ARRAY[24, 7, 25, 26, 123]),            -- Zubat
(42, 1, ARRAY[24, 7, 25, 26, 123]),            -- Golbat
(43, 1, ARRAY[2, 12, 20, 7, 121]),             -- Oddish
(44, 1, ARRAY[2, 12, 20, 7, 121]),             -- Gloom
(45, 1, ARRAY[2, 12, 20, 7, 121]),             -- Vileplume
(46, 1, ARRAY[4, 7, 20, 123, 121]),            -- Paras
(47, 1, ARRAY[4, 7, 20, 123, 121]),            -- Parasect
(48, 1, ARRAY[4, 7, 20, 123, 121]),            -- Venonat
(49, 1, ARRAY[4, 7, 20, 123, 121]),            -- Venomoth
(50, 1, ARRAY[14, 4, 31, 7, 127]),             -- Diglett
(51, 1, ARRAY[14, 4, 31, 7, 127]),             -- Dugtrio
(52, 1, ARRAY[1, 7, 27, 30, 104]),             -- Meowth
(53, 1, ARRAY[1, 7, 27, 30, 104]),             -- Persian
(54, 1, ARRAY[4, 11, 22, 7, 104]),             -- Psyduck
(55, 1, ARRAY[4, 11, 22, 7, 104]),             -- Golduck
(56, 1, ARRAY[2, 7, 27, 30, 104]),             -- Mankey
(57, 1, ARRAY[2, 7, 27, 30, 104]),             -- Primeape
(58, 1, ARRAY[3, 10, 7, 104, 137]),            -- Growlithe
(59, 1, ARRAY[3, 10, 7, 14, 104, 137]),        -- Arcanine
(60, 1, ARRAY[4, 11, 22, 7, 104]),      	  -- Poliwag
(61, 1, ARRAY[4, 11, 22, 7, 121]),       	  -- Poliwhirl
(62, 1, ARRAY[4, 11, 22, 7, 121]),       	  -- Poliwrath
(63, 1, ARRAY[18, 19, 7, 14, 128]),           -- Abra
(64, 1, ARRAY[18, 19, 7, 14, 128]),           -- Kadabra
(65, 1, ARRAY[18, 19, 7, 14, 128]),           -- Alakazam
(66, 1, ARRAY[2, 7, 27, 30, 104]),            -- Machop
(67, 1, ARRAY[2, 7, 27, 30, 104]),            -- Machoke
(68, 1, ARRAY[2, 7, 27, 30, 104]),            -- Machamp
(69, 1, ARRAY[2, 12, 20, 7, 121]),            -- Bellsprout
(70, 1, ARRAY[2, 12, 20, 7, 121]),            -- Weepinbell
(71, 1, ARRAY[2, 12, 20, 7, 121]),            -- Victreebel
(72, 1, ARRAY[8, 11, 22, 7, 121]),            -- Tentacool
(73, 1, ARRAY[8, 11, 22, 7, 121]),            -- Tentacruel
(74, 1, ARRAY[6, 14, 4, 31, 127]),            -- Geodude
(75, 1, ARRAY[6, 14, 4, 31, 127]),            -- Graveler
(76, 1, ARRAY[6, 14, 4, 31, 127]),            -- Golem
(77, 1, ARRAY[3, 10, 7, 14, 104, 137]),       -- Ponyta
(78, 1, ARRAY[3, 10, 7, 14, 104, 137]),       -- Rapidash
(79, 1, ARRAY[4, 11, 22, 7, 128]),            -- Slowpoke
(80, 1, ARRAY[4, 11, 22, 7, 128]),            -- Slowbro
(81, 1, ARRAY[13, 7, 5, 12, 123]),                 -- Magnemite
(82, 1, ARRAY[13, 7, 5, 12, 123]),                 -- Magneton
(83, 1, ARRAY[1, 7, 33, 34, 104]),                 -- Farfetch'd
(84, 1, ARRAY[6, 33, 7, 34, 104]),                 -- Doduo
(85, 1, ARRAY[6, 33, 7, 34, 104]),                 -- Dodrio
(86, 1, ARRAY[11, 7, 4, 22, 104]),                 -- Seel
(87, 1, ARRAY[11, 7, 4, 22, 104]),                 -- Dewgong
(88, 1, ARRAY[8, 7, 24, 25, 104]),                 -- Grimer
(89, 1, ARRAY[8, 7, 24, 25, 104]),                 -- Muk
(90, 1, ARRAY[11, 7, 4, 22, 104]),                 -- Shellder
(91, 1, ARRAY[11, 7, 4, 22, 104]),                 -- Cloyster
(92, 1, ARRAY[7, 24, 25, 128, 104]),               -- Gastly
(93, 1, ARRAY[7, 24, 25, 128, 104]),               -- Haunter
(94, 1, ARRAY[7, 24, 25, 128, 104]),               -- Gengar
(95, 1, ARRAY[6, 14, 4, 127, 104]),                 -- Onix
(96, 1, ARRAY[18, 19, 7, 14, 128]),                -- Drowzee
(97, 1, ARRAY[18, 19, 7, 14, 128]),                -- Hypno
(98, 1, ARRAY[11, 7, 4, 22, 104]),                 -- Krabby
(99, 1, ARRAY[11, 7, 4, 22, 104]),                 -- Kingler
(100, 1, ARRAY[13, 7, 5, 12, 123]),                -- Voltorb
(101, 1, ARRAY[13, 7, 5, 12, 123]),                -- Electrode
(102, 1, ARRAY[12, 7, 20, 21, 121]),               -- Exeggcute
(103, 1, ARRAY[12, 7, 20, 21, 121]),               -- Exeggutor
(104, 1, ARRAY[5, 7, 14, 27, 30, 127]),            -- Cubone
(105, 1, ARRAY[5, 7, 14, 27, 30, 127]),            -- Marowak
(106, 1, ARRAY[2, 7, 27, 30, 104]),           	   -- Hitmonlee
(107, 1, ARRAY[2, 7, 27, 30, 104]),          	   -- Hitmonchan
(108, 1, ARRAY[1, 7, 27, 30, 104]),                -- Lickitung
(109, 1, ARRAY[8, 7, 24, 25, 104]),                -- Koffing
(110, 1, ARRAY[8, 7, 24, 25, 104]),                -- Weezing
(111, 1, ARRAY[6, 5, 7, 14, 127]),                 -- Rhyhorn
(112, 1, ARRAY[6, 5, 7, 14, 127]),                 -- Rhydon
(113, 1, ARRAY[1, 7, 27, 30, 104]),                -- Chansey
(114, 1, ARRAY[12, 7, 20, 21, 121]),               -- Tangela
(115, 1, ARRAY[1, 7, 27, 30, 104]),                -- Kangaskhan
(116, 1, ARRAY[11, 7, 4, 22, 104]),                -- Horsea
(117, 1, ARRAY[11, 7, 4, 22, 104]),                -- Seadra
(118, 1, ARRAY[11, 7, 4, 22, 104]),                -- Goldeen
(119, 1, ARRAY[11, 7, 4, 22, 104]),                -- Seaking
(120, 1, ARRAY[11, 7, 4, 22, 104]),                -- Staryu
(121, 1, ARRAY[11, 7, 4, 22, 128]),        		   -- Starmie
(122, 1, ARRAY[14, 7, 18, 19, 128]),               -- Mr. Mime
(123, 1, ARRAY[4, 7, 33, 34, 104]),                -- Scyther
(124, 1, ARRAY[15, 7, 16, 17, 104]),               -- Jynx
(125, 1, ARRAY[13, 7, 5, 12, 123]),                -- Electabuzz
(126, 1, ARRAY[10, 7, 3, 14, 104, 137]),           -- Magmar
(127, 1, ARRAY[4, 7, 33, 34, 104]),                -- Pinsir
(128, 1, ARRAY[1, 7, 27, 30, 104]),                -- Tauros
(129, 1, ARRAY[11, 7, 4, 22, 104]),                -- Magikarp
(130, 1, ARRAY[11, 7, 6, 33, 123]),                -- Gyarados
(131, 1, ARRAY[11, 7, 4, 22, 104]),                -- Lapras
(132, 1, ARRAY[1, 7, 27, 30, 104]),                -- Ditto
(133, 1, ARRAY[1, 7, 27, 30, 104]),                -- Eevee
(134, 1, ARRAY[11, 7, 4, 22, 104]),                -- Vaporeon
(135, 1, ARRAY[13, 7, 5, 12, 104]),                -- Jolteon
(136, 1, ARRAY[10, 7, 3, 14, 104]),                -- Flareon
(137, 1, ARRAY[1, 7, 27, 30, 104]),                -- Porygon
(138, 1, ARRAY[6, 11, 7, 4, 104]),                 -- Omanyte
(139, 1, ARRAY[6, 11, 7, 4, 104]),                 -- Omastar
(140, 1, ARRAY[6, 11, 7, 4, 104]),                 -- Kabuto
(141, 1, ARRAY[6, 11, 7, 4, 104]),                 -- Kabutops
(142, 1, ARRAY[6, 3, 7, 34, 104]),                 -- Aerodactyl
(143, 1, ARRAY[1, 7, 27, 30, 143]),                -- Snorlax
(144, 1, ARRAY[15, 3, 7, 16, 17]),                 -- Articuno
(145, 1, ARRAY[13, 3, 7, 5, 12, 123]),             -- Zapdos
(146, 1, ARRAY[10, 3, 7, 3, 14, 104, 137]),        -- Moltres
(147, 1, ARRAY[16, 7, 81, 122, 123, 121]),         -- Dratini
(148, 1, ARRAY[16, 7, 81, 122, 123, 121]),         -- Dragonair
(149, 1, ARRAY[16, 3, 7, 81, 122, 123, 121]),      -- Dragonite
(150, 1, ARRAY[14, 7, 18, 19, 128, 145]),          -- Mewtwo
(151, 1, ARRAY[14, 7, 18, 19, 128, 137, 145]);     -- Mew
""",
"""
-- Jotave tem um Charizard e 5 pokémons aleatorios
INSERT INTO Equipe (id_equipe, id_treinador, id_pokemons) VALUES
(1, 1, ARRAY[6, 25, 2, 45, 77, 12]);
""",
"""
-- Panza tem um Dragonite e 5 pokémons aleatórios
INSERT INTO Equipe (id_equipe, id_treinador, id_pokemons) VALUES
(2, 2, ARRAY[149, 3, 20, 50, 90, 130]);
""",
"""
-- Barbara com pokémons cor de rosa
INSERT INTO Equipe (id_equipe, id_treinador, id_pokemons) VALUES
(3, 3, ARRAY[36, 39, 40, 113, 80, 108]); 
""",
"""
-- Daniel com pokémons competitivamente fortes
INSERT INTO Equipe (id_equipe, id_treinador, id_pokemons) VALUES
(4, 4, ARRAY[65, 130, 143, 135, 94, 121]);
""",
"""
-- Raquel com pokémons fracos contra o Lider de Ginasio Giovanni
INSERT INTO Equipe (id_equipe, id_treinador, id_pokemons) VALUES
(5, 5, ARRAY[25, 37, 126, 109, 101, 58]);  
""",
"""
-- Batalhas dos treinadores contra os membros da liga
INSERT INTO Batalha (id_treinador_1, id_treinador_2, vencedor) VALUES
(1, 6, 1),   -- Jotave vs Lorelei
(1, 7, 7),   -- Jotave vs Bruno

(2, 6, 2),   -- Panza vs Lorelei
(2, 7, 2),   -- Panza vs Bruno
(2, 8, 2),   -- Panza vs Agatha
(2, 9, 9),   -- Panza vs Lance

(3, 6, 3),   -- Barbara vs Lorelei
(3, 7, 3),   -- Barbara vs Bruno
(3, 8, 8),   -- Barbara vs Agatha

(4, 6, 4),   -- Daniel vs Lorelei
(4, 7, 4),   -- Daniel vs Bruno
(4, 8, 4),   -- Daniel vs Agatha
(4, 9, 4),   -- Daniel vs Lance

(5, 6, 5);   -- Raquel vs Lorilei
""",
"""
-- Batalhas dos 5 treinadores batalhando entre eles
INSERT INTO Batalha (id_treinador_1, id_treinador_2, vencedor) VALUES
(1, 2, 1),    -- Jotave vs Panza
(2, 3, 3),    -- Panza vs Barbara
(3, 4, 4),    -- Barbara vs Daniel
(4, 5, 4),    -- Daniel vs Raquel
(5, 1, 1);    -- Raquel vs Jotave
"""
]

# Executando os comandos
for command in sql_commands:
    cursor.execute(command)
    
print("Inserções criadas com sucesso!")

Inserções criadas com sucesso!


In [37]:
# Criação de Index no banco de dados
sql_commands = [
"""
-- Índice para acelerar a busca de treinadores e Pokémon por atributos de ataque (Select 2)
CREATE INDEX idx_pokemon_ataque ON Pokemon (ataque);
""",
"""
CREATE INDEX idx_pokemon_ataque_especial ON Pokemon (ataque_especial);
""",
"""
-- Índice para a busca de tipos na Pokedex (Select 3)
CREATE INDEX idx_pokedex_id_tipo ON Pokedex USING gin (id_tipo);
""",
"""
-- Índice para busca por tipos e golpes aprendidos (Select 5)
CREATE INDEX idx_pokemon_id_tipo ON Pokemon USING gin (id_tipo);
""",
"""
-- Índice para encontrar Pokémon que evoluem com item (Select 6)
CREATE INDEX idx_pokemon_evolui_com_pedra ON Pokemon (evolui_com_pedra);
""",
"""
-- Índice para buscar Pokémon com mais de um tipo (Select 7)
CREATE INDEX idx_pokemon_tipos ON Pokemon USING gin (id_tipo);
""",
"""
-- Índice para buscar treinadores com 8 insígnias (Select 8)
CREATE INDEX idx_treinador_id_insignias ON Treinador USING gin (id_insignias);
""",
"""
-- Índice para golpes com precisão < 100 (Select 9)
CREATE INDEX idx_golpe_precisao ON Golpe (precisao);
""",
"""
-- Índice para Pokémon com nível maior ou igual a 50 (Select 10)
CREATE INDEX idx_pokemon_nivel ON Pokemon (nivel);
""",
"""
-- Índice para golpes comuns entre Pokémon (Select 12)
CREATE INDEX idx_pokemon_id_golpes ON Pokemon USING gin (id_golpes);
"""
]

# Executando os comandos
for command in sql_commands:
    cursor.execute(command)

print("Indexações criadas com sucesso!")

Indexações criadas com sucesso!


In [38]:
# Criação de Views no banco de dados
sql_commands = [
"""
CREATE OR REPLACE VIEW treinador_pokemon AS
SELECT 
	t.id_treinador, 
    p.id_pokedex_pokemon, 
    p.id_instancia_pokemon
FROM 
	Treinador t 
JOIN 
	Pokemon p ON  p.id_treinador = t.id_treinador;
""",
"""
CREATE OR REPLACE VIEW interface_de_batalha AS
SELECT 
    t.nome AS nome_treinador,  
    p.nome AS nome_pokemon,    
    p.hp, 
    p.nivel, 
    p.id_golpes
FROM 
    Treinador t 
JOIN 
    Pokemon p ON p.id_treinador = t.id_treinador;
""",
"""
CREATE OR REPLACE VIEW pokemon_com_tipos AS
SELECT 
    p.id_pokedex_pokemon,
    p.nome,
    ARRAY_AGG(DISTINCT t.id_tipo) AS id_tipo,
    STRING_AGG(DISTINCT t.nome, ', ') AS tipo
FROM 
    Pokemon p
LEFT JOIN Tipo t ON t.id_tipo = ANY(p.id_tipo)
GROUP BY p.id_pokedex_pokemon, p.nome;
"""
]

# Executando os comandos
for command in sql_commands:
    cursor.execute(command)

print("Views criadas com sucesso!")

Views criadas com sucesso!


In [39]:
# Criação de Procedures e Functions no banco de dados
sql_commands = [
"""
CREATE OR REPLACE FUNCTION captura_pokemon(
    id_pokedex_pokemon1 INTEGER,
    id_instancia_pokemon1 INTEGER,
    id_treinador1 INTEGER
) RETURNS BOOLEAN AS $$
DECLARE
    possui_pokebola BOOLEAN;
BEGIN
    SELECT EXISTS (
        SELECT 1
        FROM unnest((SELECT id_itens FROM Treinador WHERE id_treinador = id_treinador1)) AS item
        WHERE item IN (SELECT id_item FROM Item_Pokebola)
    ) INTO possui_pokebola;

    IF possui_pokebola AND EXISTS (
        SELECT 1
        FROM Pokemon
        WHERE id_pokedex_pokemon = id_pokedex_pokemon1
          AND id_instancia_pokemon = id_instancia_pokemon1
          AND id_treinador IS NULL
    ) THEN
        UPDATE Pokemon
        SET id_treinador = id_treinador1
        WHERE id_pokedex_pokemon = id_pokedex_pokemon1
          AND id_instancia_pokemon = id_instancia_pokemon1;

        RETURN TRUE;
    ELSE
        RETURN FALSE; 
    END IF;
END;
$$ LANGUAGE plpgsql;
""",
"""
-----------------------------------------------------------
CREATE OR REPLACE FUNCTION registrar_batalha(
    id_treinador_1 INTEGER,
    id_treinador_2 INTEGER,
    vencedor integer
) RETURNS VOID AS $$
BEGIN
    IF vencedor IN (
        SELECT nome FROM Treinador WHERE id_treinador IN (id_treinador_1, id_treinador_2)
    ) THEN
        INSERT INTO Batalha (id_treinador_1, id_treinador_2, resultado)
    	VALUES (id_treinador_1, id_treinador_2, vencedor);
    END IF;
END;
$$ LANGUAGE plpgsql;
""",
"""
-----------------------------------------------------------
-- se tem 4 golpes não pode colocar mais
CREATE OR REPLACE FUNCTION adicionar_golpe(
    id_pokedex_pokemon1 INTEGER,
    id_instancia_pokemon1 INTEGER,
    id_golpe INTEGER
) RETURNS VOID AS $$
BEGIN
    IF id_golpe IN (
        SELECT unnest(id_golpes)
        FROM Pode_Aprender
        WHERE id_pokedex_pokemon = id_pokedex_pokemon1
    ) AND NOT EXISTS (
        SELECT 1
        FROM Pokemon
        WHERE id_pokedex_pokemon = id_pokedex_pokemon1
          AND id_instancia_pokemon = id_instancia_pokemon1
          AND id_golpe = ANY(id_golpes)
    ) AND (
        SELECT array_length(id_golpes, 1)
        FROM Pokemon
        WHERE id_pokedex_pokemon = id_pokedex_pokemon1
          AND id_instancia_pokemon = id_instancia_pokemon1
    ) < 4 THEN
        UPDATE Pokemon
        SET id_golpes = array_append(id_golpes, id_golpe)
        WHERE id_pokedex_pokemon = id_pokedex_pokemon1
          AND id_instancia_pokemon = id_instancia_pokemon1;
    END IF;
END;
$$ LANGUAGE plpgsql;
""",
"""
-----------------------------------------------------------
-- Função para subir de nível e ajustar os atributos
CREATE OR REPLACE FUNCTION subir_de_nivel(
    p_id_pokedex_pokemon INTEGER,
    p_id_instancia_pokemon INTEGER
)
RETURNS VARCHAR AS $$
DECLARE
    nivel_atual INTEGER;
    nivel_maximo CONSTANT INTEGER := 100;
BEGIN
    SELECT nivel INTO nivel_atual
    FROM Pokemon
    WHERE id_pokedex_pokemon = p_id_pokedex_pokemon 
      AND id_instancia_pokemon = p_id_instancia_pokemon;

    IF nivel_atual >= nivel_maximo THEN
        RETURN 'O Pokémon não pode mais subir de nível, pois está no nível máximo.';
    END IF;

    UPDATE Pokemon
    SET 
        nivel = nivel + 1,
        hp = hp + 2,
        ataque = ataque + 2,
        defesa = defesa + 2,
        ataque_especial = ataque_especial + 2,
        defesa_especial = defesa_especial + 2,
        velocidade = velocidade + 2
    WHERE id_pokedex_pokemon = p_id_pokedex_pokemon 
      AND id_instancia_pokemon = p_id_instancia_pokemon;

    RETURN FORMAT('O Pokémon subiu para o nível %s.', nivel_atual + 1);
END;
$$ LANGUAGE plpgsql;
""",
"""
------------------------------------------------------------------
CREATE OR REPLACE FUNCTION transferir_pokemon(
    id_pokedex_pokemon1 INTEGER,
    id_instancia_pokemon1 INTEGER,
    id_treinador_origem INTEGER,
    id_treinador_destino INTEGER
) RETURNS TEXT AS $$
DECLARE
    treinador_atual INTEGER;
BEGIN
    SELECT id_treinador
    INTO treinador_atual
    FROM Pokemon
    WHERE id_pokedex_pokemon = id_pokedex_pokemon1
      AND id_instancia_pokemon = id_instancia_pokemon1;

    IF treinador_atual IS NULL THEN
        RETURN 'Erro: Este Pokémon não pertence a nenhum treinador.';
    ELSIF treinador_atual != id_treinador_origem THEN
        RETURN 'Erro: O Pokémon não pertence ao treinador de origem.';
    END IF;

    IF NOT EXISTS (
        SELECT 1
        FROM Treinador
        WHERE id_treinador = id_treinador_destino
    ) THEN
        RETURN 'Erro: Treinador de destino não encontrado.';
    END IF;

    UPDATE Pokemon
    SET id_treinador = id_treinador_destino
    WHERE id_pokedex_pokemon = id_pokedex_pokemon1
      AND id_instancia_pokemon = id_instancia_pokemon1;

    RETURN FORMAT(
        'Transferência concluída! Pokémon %s (instância %s) agora pertence ao Treinador %s.',
        id_pokedex_pokemon1, id_instancia_pokemon1, id_treinador_destino
    );
END;
$$ LANGUAGE plpgsql;
""",
"""
------------------------------------------------------------------
CREATE OR REPLACE FUNCTION adicionar_pokemon_equipe(
    treinador_id INTEGER,
    pokemon_id INTEGER
)
RETURNS VARCHAR AS $$
DECLARE
    mensagem VARCHAR;
BEGIN
    INSERT INTO Equipe (id_treinador, id_pokemons)
    VALUES (treinador_id, ARRAY[pokemon_id])
    ON CONFLICT (id_treinador)
    DO UPDATE 
    SET id_pokemons = array_append(Equipe.id_pokemons, pokemon_id);

    mensagem := 'Pokémon '  ||pokemon_id||  ' foi adicionado à equipe do treinador '  ||treinador_id||  '.';

    RETURN mensagem;

END;
$$ LANGUAGE plpgsql;
"""
]

# Executando os comandos
for command in sql_commands:
    cursor.execute(command)

print("Procedures/Functions criadas com sucesso!")

Procedures/Functions criadas com sucesso!


In [40]:
# Criação de Triggers no banco de dados
sql_commands = [
"""
CREATE OR REPLACE FUNCTION verifica_equipe_apos_troca()
RETURNS TRIGGER AS $$
BEGIN
    -- Verifica se o Pokémon antigo faz parte de uma equipe
    IF EXISTS (
        SELECT 1
        FROM Equipe
        WHERE id_pokedex_pokemon = OLD.id_pokedex_pokemon
          AND id_instancia_pokemon = OLD.id_instancia_pokemon
    ) THEN
        -- Remove o Pokémon da equipe
        DELETE FROM Equipe
        WHERE id_pokedex_pokemon = OLD.id_pokedex_pokemon
          AND id_instancia_pokemon = OLD.id_instancia_pokemon;
    END IF;

    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

-- Trigger para remover Pokémon de uma equipe após a troca
CREATE TRIGGER trg_remove_pokemon_da_equipe
AFTER UPDATE OF id_treinador ON Pokemon
FOR EACH ROW
WHEN (OLD.id_treinador IS DISTINCT FROM NEW.id_treinador)
EXECUTE FUNCTION verifica_equipe_apos_troca();
""",
"""
------------------------------------------------------------------
CREATE OR REPLACE FUNCTION verifica_participacao_liga()
RETURNS TRIGGER AS $$
BEGIN
    IF (SELECT COUNT(*) FROM treinador T JOIN insignia I on I.id_insignia = ANY(T.id_insignias) WHERE id_treinador = NEW.id_treinador) < 8 THEN
        RAISE EXCEPTION 'Treinador precisa ter as 8 insinias para participar da liga.';
    END IF;
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER trg_verifica_participacao_liga
BEFORE INSERT ON liga
FOR EACH ROW
EXECUTE FUNCTION verifica_participacao_liga();
""",
"""
------------------------------------------------------------------
CREATE OR REPLACE FUNCTION verifica_limite_equipe()
RETURNS TRIGGER AS $$
BEGIN
    IF (SELECT array_length(id_pokemons, 1) FROM Equipe WHERE id_treinador = NEW.id_treinador) >= 6 THEN
        RAISE EXCEPTION 'Treinador já possui o limite de 6 Pokémon na equipe.';
    END IF;
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE OR REPLACE TRIGGER trg_verifica_limite_equipe
BEFORE UPDATE OR INSERT ON Equipe
FOR EACH ROW
EXECUTE FUNCTION verifica_limite_equipe(); 
"""
]

# Executando os comandos
for command in sql_commands:
    cursor.execute(command)
    
print("Triggers criadas com sucesso!")

Triggers criadas com sucesso!


In [41]:
# Consultas no banco de dados
sql_commands = [
"""
-- Select 1
SELECT 
    t.nome AS vencedor, 
    COUNT(*) AS vitorias 
FROM 
    Batalha b JOIN Treinador t
on 
    b.vencedor = t.id_treinador
GROUP BY 
    t.nome
ORDER BY 
    vitorias DESC
LIMIT 10;
""",
"""
-- Select 2
SELECT 
    t.nome AS treinador, 
    STRING_AGG(DISTINCT p.nome, ', '),
    ARRAY_AGG(DISTINCT p.ataque),
    ARRAY_AGG(DISTINCT p.ataque_especial)
FROM 
    Treinador t 
JOIN 
    Equipe e 
ON 
    t.id_treinador = e.id_treinador
JOIN 
    Pokemon p 
ON 
    p.id_treinador = t.id_treinador
where
    p.ataque >= 100 or p.ataque_especial >= 100
GROUP BY 
    T.nome;
""",
"""  
-- Select 3
SELECT 
    STRING_AGG(DISTINCT t.nome, ', ') AS tipos, 
    COUNT(*) AS quantidade 
FROM 
    Pokedex p
JOIN 
    Tipo t ON t.id_tipo = ANY(p.id_tipo)
GROUP BY 
    p.id_tipo
ORDER BY 
    quantidade DESC;
""",
"""
-- SELECT 4
SELECT 
    t.nome AS tipo, 
    COUNT(g.id_golpe) AS quantidade_golpes
FROM 
    Golpe g
JOIN 
    Tipo t ON g.id_tipo = t.id_tipo
GROUP BY 
    t.nome
ORDER BY 
    quantidade_golpes DESC;
""",
""" 
-- Select 5
SELECT
    p.nome AS pokemon, 
    t.nome AS tipo, 
    g.nome AS golpe,
    g.poder
FROM 
    Pokemon P
JOIN 
    Tipo T 
ON 
    T.id_tipo = ANY(P.id_tipo)
JOIN 
    pode_aprender PA 
on 
    PA.id_pokedex_pokemon = P.id_pokedex_pokemon
JOIN 
    golpe G 
on 
    G.id_golpe = ANY(PA.id_golpes)
WHERE 
    T.nome = 'Dragon' and p.id_instancia_pokemon = 1
ORDER BY 
    p.nome, t.nome, g.nome;
""",
"""
-- Select 6
SELECT 
    DISTINCT p.nome AS pokemon, 
    i.nome AS item 
FROM 
    Item i
JOIN 
    Item_Evolucao ie 
ON 
    i.id_item = ie.id_item and i.nome != 'Rare Candy'
JOIN
    Pokemon p
ON 
    p.evolui_com_pedra = TRUE AND p.id_tipo[1] = ANY(ie.id_tipo)
WHERE
	p.nome != 'Eevee'
order by
	i.nome;
""",
"""
-- Select 7
SELECT 
    p.nome, 
    STRING_AGG(DISTINCT t.nome, ', ')
FROM 
    Pokemon p
JOIN 
    Tipo t ON t.id_tipo = ANY(p.id_tipo)
GROUP BY 
    p.nome
HAVING 
    COUNT(t.id_tipo) > 1;
""",
"""
-- Select 8
SELECT 
    t.nome, 
    STRING_AGG(i.nome, ', ')
FROM 
    Treinador t
JOIN
	Insignia i
ON
	i.id_insignia = any(t.id_insignias)
WHERE 
    array_length(id_insignias, 1) = 8 and T.eh_campeao = FALSE and T.id_treinador not in (6,7,8,9)
GROUP BY 
    t.nome;
""",
""" 
-- Select 9
SELECT 
    t.nome AS treinador,
    STRING_AGG(g.nome || ' (Poder: ' || g.poder || ', Precisão: ' || g.precisao || ')', ', ') AS golpes
FROM 
    Golpe g
JOIN 
    Pokemon p 
ON 
	g.id_golpe = ANY(p.id_golpes)
JOIN 
    Treinador t 
ON 
	t.id_treinador = p.id_treinador AND t.id_treinador NOT IN (6,7,8,9)
WHERE 
    g.precisao < 100
GROUP BY 
    t.nome
ORDER BY 
    MAX(g.poder) DESC;
""",
"""
-- Select 10
SELECT 
    t.nome AS treinador, 
    STRING_AGG(p.nome || ' (Nível: ' || p.nivel || ')', ', ') AS pokemon
FROM 
    Treinador t
JOIN 
    Pokemon p 
ON 
	t.id_treinador = p.id_treinador
WHERE 
    p.nivel >= 50 AND t.id_treinador NOT IN (6,7,8,9)
GROUP BY 
    t.nome
ORDER BY 
    MAX(p.nivel) DESC;
""",
"""
-- Select 11
SELECT 
    t.nome AS treinador, 
    COUNT(p.id_pokedex_pokemon) AS total_pokemon,
    STRING_AGG(DISTINCT p.nome, ', ') AS tipos
FROM 
    Treinador t
JOIN 
    Pokemon p 
ON 
	t.id_treinador = p.id_treinador AND t.id_treinador NOT IN (6,7,8,9)
GROUP BY 
    t.nome
ORDER BY 
    total_pokemon DESC;
""",
"""
-- Select 12
SELECT 
    p1.nome AS pokemon1, 
    p2.nome AS pokemon2, 
    STRING_AGG(g.nome, ', ') AS golpes_comuns
FROM 
    Pokemon p1
JOIN 
    Pokemon p2 ON p1.id_pokedex_pokemon != p2.id_pokedex_pokemon
JOIN 
    Golpe g 
ON 
	g.id_golpe = ANY(p1.id_golpes) AND g.id_golpe = ANY(p2.id_golpes)
WHERE 
    p1.id_tipo = p2.id_tipo and 10 = ANY(p1.id_tipo)
GROUP BY p1.nome, p2.nome;
"""
]

# Executando os comandos
for command in sql_commands:
    cursor.execute(command)

print("Consultas criadas com sucesso!")

Consultas criadas com sucesso!


In [42]:
# Fechando a conexão inicial
cursor.close()
connection.close()